<a href="https://colab.research.google.com/github/Risskr/Stock-App/blob/Working/StocksApp_DashApp_lowpoly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Section 1: Set Up**


In [1]:
# #-----refesh python script-------#
# get_ipython().kernel.do_shutdown(restart=True)

# #----GPU Setup------#
# #Check GPUs
# !nvidia-smi

# #load cudf to use GPUs for analysis
# %load_ext cudf.pandas


#------Imports--------#
import pandas as pd
import datetime
import pickle
import numpy as np
import requests
import time
import os


#-------Froms-------#
from tqdm.notebook import tqdm
from datetime import datetime, timedelta


# #------Force Connect MyDrive-------#
# from google.colab import drive
# drive.mount('/content/drive')


# **Section 2: Stock Data**

## Ensure the last 6 months of EODHD Data is available

In [26]:
# Ensure the last 6 months of EODHD Data is available
"""
Returns:
nasdaq_df:
  <class 'pandas.core.frame.DataFrame'>
  RangeIndex: 627225 entries, 0 to 627224
  Data columns (total 8 columns):
  #   Column          Non-Null Count   Dtype
  ---  ------          --------------   -----
  0   date            627225 non-null  datetime64[ns]
  1   ticker          627098 non-null  object
  2   open            627225 non-null  float64
  3   high            627225 non-null  float64
  4   low             627225 non-null  float64
  5   close           627225 non-null  float64
  6   adjusted_close  627225 non-null  float64
  7   volume          627225 non-null  float64
"""
#API_KEY = 'demo'  # Replace with your API key if not using demo
API_KEY = '68433aff09ea73.10710364'
EXCHANGE = 'NASDAQ'
DAYS_BACK = 180  # Approx. 6 months
SAVE_PATH = '/content/drive/MyDrive/Colab Notebooks/nasdaq_bulk_eod.csv'
MAX_CALLS_PER_RUN = 200  # Use 1–5 for the free tier
SECONDS_BETWEEN_CALLS = 0  # Add delay to be respectful

# Generate past 6 months of weekdays
today = datetime.utcnow().date()
dates = [today - timedelta(days=i) for i in range(DAYS_BACK)]
dates = sorted([d for d in dates if d.weekday() < 5])  # Keep only weekdays

# Load already-downloaded dates if file exists
downloaded_dates = set()
if os.path.exists(SAVE_PATH):
    df_existing = pd.read_csv(SAVE_PATH)
    downloaded_dates = set(pd.to_datetime(df_existing['date']).dt.date)
# Filter to only dates we haven’t downloaded
pending_dates = [d for d in dates if d not in downloaded_dates]

# Prepare data holder
all_data = []

for i, date in enumerate(pending_dates[:MAX_CALLS_PER_RUN]):
    date_str = date.strftime('%Y-%m-%d')
    url = f'https://eodhd.com/api/eod-bulk-last-day/{EXCHANGE}?api_token={API_KEY}&fmt=json&date={date_str}'
    print(f"[{i+1}] Fetching {date_str}...")

    try:
        response = requests.get(url)
        response.raise_for_status()
        day_data = response.json()

        for entry in day_data:
            all_data.append({
                'date': entry.get('date'),
                'ticker': entry.get('code'),
                'open': entry.get('open'),
                'high': entry.get('high'),
                'low': entry.get('low'),
                'close': entry.get('close'),
                'adjusted_close': entry.get('adjusted_close'),
                'volume': entry.get('volume'),
            })

        # Respect the delay
        if i < MAX_CALLS_PER_RUN - 1:
            time.sleep(SECONDS_BETWEEN_CALLS)

    except Exception as e:
        print(f"⚠️ Error on {date_str}: {e}")

# Append or save the new data
if all_data:
    df_new = pd.DataFrame(all_data)
    if os.path.exists(SAVE_PATH):
        df_existing = pd.read_csv(SAVE_PATH)
        df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    else:
        df_combined = df_new

    df_combined.to_csv(SAVE_PATH, index=False)
    print(f"✅ Data for {len(all_data)} entries added to {SAVE_PATH}")

else:
    print("⚠️ No new data fetched.")


# Import CSV into code
nasdaq_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/nasdaq_bulk_eod.csv")

# Save the latest date in YYYYMMDD format
nasdaq_df['date'] = pd.to_datetime(nasdaq_df['date'])
latest_date_nasdaq_data = nasdaq_df['date'].max().strftime('%Y%m%d')

⚠️ No new data fetched.


## Get list of Tickers with Min Market Cap and populate screener_data_df

In [95]:
# Get list of Tickers with a min market cap and a list of common stocks
"""
Returns:
screener_data_df:
  <class 'pandas.core.frame.DataFrame'>
  RangeIndex: 262 entries, 0 to 261
  Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype
  ---  ------                 --------------  -----
 0   code                   262 non-null    object
 1   name                   262 non-null    object
 2   last_day_data_date     262 non-null    object
 3   adjusted_close         262 non-null    float64
 4   refund_1d              262 non-null    float64
 5   refund_1d_p              262 non-null    float64
 6   refund_5d              262 non-null    float64
 7   refund_5d_p            262 non-null    float64
 8   exchange               262 non-null    object
 9   currency_symbol        262 non-null    object
 10  market_capitalization  262 non-null    int64
 11  earnings_share         262 non-null    float64
 12  dividend_yield         145 non-null    float64
 13  sector                 262 non-null    object
 14  industry               262 non-null    object
 15  avgvol_1d              262 non-null    int64
 16  avgvol_200d            262 non-null    float64
 17  Country                262 non-null    object
 18  Exchange               262 non-null    object
 19  Currency               262 non-null    object
 20  Type                   262 non-null    object
 21  last_day_change        262 non-null    float64

"""

# -------------------- CONFIG --------------------
API_KEY = '68433aff09ea73.10710364'  # Replace with your EODHD key
MIN_MARKET_CAP = 10_000_000_000  # Changeable: $1B, $10B, etc.
EXCHANGE = 'NASDAQ'
RESULTS_PER_PAGE = 500  # Max per EODHD API
# ------------------------------------------------

def get_filtered_nasdaq_stocks(api_key, min_cap, exchange="NASDAQ"):
    all_data = []
    offset = 0

    while True:
        url = (
            "https://eodhd.com/api/screener"
            f"?api_token={api_key}"
            f"&filters=["
            f'["exchange","=","{exchange}"],'
            #f'["type","=","Common Stock"],'
            f'["market_capitalization",">=",{min_cap}]'
            f"]"
            f"&sort=market_capitalization.desc"
            f"&limit={RESULTS_PER_PAGE}&offset={offset}&fmt=json"
        )

        response = requests.get(url)
        result = response.json()
        batch = result.get("data", [])

        if not batch:
            break

        all_data.extend(batch)
        offset += RESULTS_PER_PAGE

    return pd.DataFrame(all_data)

# Run filter to get screener_data_df
screener_data_df = get_filtered_nasdaq_stocks(API_KEY, MIN_MARKET_CAP)

# Get exchange symbol list with type
meta_url = f'https://eodhd.com/api/exchange-symbol-list/NASDAQ?api_token={API_KEY}&fmt=json'
meta_df = pd.DataFrame(requests.get(meta_url).json())
common_df = meta_df[meta_df['Type'] == 'Common Stock'].copy() # Add .copy() to avoid SettingWithCopyWarning

# Combine screener_data_df with relevant columns from common_df
screener_data_df = pd.merge(
    screener_data_df,
    common_df[['Code', 'Country', 'Exchange', 'Currency', 'Type']],
    left_on='code',
    right_on='Code',
    how='inner'
).drop('Code', axis=1)

# --- Calculate Daily Change ---
# Ensure 'date' column is datetime and sort by date
nasdaq_df['date'] = pd.to_datetime(nasdaq_df['date'])
temp_nasdaq_sorted = nasdaq_df.sort_values(by=['ticker', 'date']).copy()

# Calculate the previous day's adjusted close for each ticker
temp_nasdaq_sorted['prev_adjusted_close'] = temp_nasdaq_sorted.groupby('ticker')['adjusted_close'].shift(1)

# Calculate the daily change as a percentage
temp_nasdaq_sorted['daily_change'] = ((temp_nasdaq_sorted['adjusted_close'] - temp_nasdaq_sorted['prev_adjusted_close']) / temp_nasdaq_sorted['prev_adjusted_close']) * 100

# Get the last day's data for each ticker
last_day_data_per_ticker = temp_nasdaq_sorted.groupby('ticker').tail(1).copy()

# Create a dictionary of the last day's change for each ticker
last_day_changes = last_day_data_per_ticker.set_index('ticker')['daily_change'].to_dict()

# Add the last day change to the screener_data_df
screener_data_df['last_day_change'] = screener_data_df['code'].map(last_day_changes)

## Filter nasdaq data for screener_data_df and type: common

In [96]:
# prompt: I want to filter the Nasdaq stocks to only include tickers that are part of the common_df and the min_market_cap_df. Same this df as a new variable
# Load your EOD data
from datetime import datetime, timedelta
# nasdaq_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/nasdaq_bulk_eod.csv")
nasdaq_df['date'] = pd.to_datetime(nasdaq_df['date'])

# Now filter nasdaq_df using the combined dataframe and type 'Common Stock'
filtered_nasdaq_df = nasdaq_df[
    nasdaq_df['ticker'].isin(screener_data_df[screener_data_df['Type'] == 'Common Stock']['code'])
]

## Function: Correlation Coeficient for entire stock data set

In [73]:
#return six month and three month spearman correlations for all unique pairs of stocks

import pandas as pd
from datetime import datetime # Import datetime
from tqdm.notebook import tqdm # Import tqdm

# ## Function: Correlation Coeficient for entire stock data set
def calculate_lagged_correlation(df, lag_days, range_months):
  """
  Calculates the pairwise spearman correlation coefficient between all stocks
  in a DataFrame for a specified period with a given lag.
  The start date is calculated by subtracting range_months from today's date.

  Args:
    df: DataFrame with 'date', 'ticker', and 'adjusted_close' columns.
    lag_days: The number of days to lag the second stock's data.
    range_months: The number of months to include in the analysis period.

  Returns:
    correlation_matrix: A pandas DataFrame with the following
      Index: Tickers
      Columns: Tickers
      Values: Spearman correlation coefficients between stocks
  """
  # Calculate the end date (today's date)
  end_datetime = datetime.now()

  # Calculate the start date for the specified period by subtracting range_months
  start_datetime = end_datetime - pd.DateOffset(months = range_months)


  # Ensure the 'date' column is in datetime format
  df['date'] = pd.to_datetime(df['date'])

  # Filter the DataFrame for the specified date range
  filtered_df = df[(df['date'] >= start_datetime) & (df['date'] <= end_datetime)].copy()

  # Filter out rows where volume is 0
  filtered_df = filtered_df[filtered_df['volume'] > 0].copy()

  # Get unique tickers in the filtered data
  tickers = filtered_df['ticker'].unique()

  #breakpoint()

  # Create an empty DataFrame to store correlation results
  correlation_matrix = pd.DataFrame(index=tickers, columns=tickers, dtype=float)

  # Iterate through all pairs of tickers with a progress bar
  for ticker_a in tqdm(tickers, desc="Calculating correlations"): # Add tqdm here

    # Extract data for each ticker and name the series for clarity
    stock_a_data = filtered_df[filtered_df['ticker'] == ticker_a].set_index('date')['adjusted_close']
    #breakpoint()

    for ticker_b in tickers:
      if ticker_a != ticker_b:
        # Extract data for each ticker and name the series for clarity
        stock_b_data = filtered_df[filtered_df['ticker'] == ticker_b].set_index('date')['adjusted_close']
        #breakpoint()

        # Align the dataframes based on the date index
        # Suffixes will be applied to the 'adjusted_close' column name
        aligned_data = pd.merge(stock_a_data, stock_b_data,
                                left_index=True, right_index=True,
                                how='inner', suffixes=('_A', '_B'))

        # Apply the lag to stock_b_data, referencing the suffixed column name
        lagged_stock_b_data = aligned_data['adjusted_close_B'].shift(lag_days)

        #breakpoint()

        # Calculate correlation, dropping NaN values
        # Using Spearman method and minimum periods
        #correlation = aligned_data['adjusted_close_A'].corr(lagged_stock_b_data, method='spearman', min_periods=100)

        correlation = aligned_data['adjusted_close_A'].corr(lagged_stock_b_data, method='spearman')


        # Store the correlation in the matrix
        correlation_matrix.loc[ticker_a, ticker_b] = correlation

  return correlation_matrix


# Check GoogleDrive for latest_date_nasdaq_data, in not run the correlation function
"""
Returns:
  three_month_spearman_lagged_correlations and six_month_spearman_lagged_correlations: A pandas DataFrame with the following
    Index: Tickers
    Columns: Tickers
    Values: Spearman correlation coefficients between stocks
"""
import pandas as pd
import os
from datetime import datetime, timedelta

# Define the file paths in Google Drive using the determined date
three_month_file = f'/content/drive/MyDrive/Colab Notebooks/Correlation_Data/{latest_date_nasdaq_data}_three_month_spearman_lagged_correlation.csv'
six_month_file = f'/content/drive/MyDrive/Colab Notebooks/Correlation_Data/{latest_date_nasdaq_data}_six_month_spearman_lagged_correlation.csv'

# Check if files exist and load them
try:
    three_month_spearman_lagged_correlations = pd.read_csv(three_month_file, index_col=0)
    print(f"Loaded existing file: {three_month_file}")
    six_month_spearman_lagged_correlations = pd.read_csv(six_month_file, index_col=0)
    print(f"Loaded existing file: {six_month_file}")

except FileNotFoundError:
    print("One or both correlation files not found. Calculating correlations...")
    # Calculate correlations if files don't exist
    # Ensure 'filtered_nasdaq_df' is defined from the preceding code
    if 'filtered_nasdaq_df' in locals():
        three_month_spearman_lagged_correlations = calculate_lagged_correlation(filtered_nasdaq_df, lag_days=1, range_months=3)
        six_month_spearman_lagged_correlations = calculate_lagged_correlation(filtered_nasdaq_df, lag_days=1, range_months=6)

        three_month_spearman_lagged_correlations.to_csv(three_month_file)
        print(f"Saved calculated file: {three_month_file}")
        six_month_spearman_lagged_correlations.to_csv(six_month_file)
        print(f"Saved calculated file: {six_month_file}")
    else:
        print("Error: 'filtered_nasdaq_df' is not defined. Please ensure the preceding code ran correctly.")

One or both correlation files not found. Calculating correlations...


/tmp/ipython-input-73-3716170834.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Calculating correlations:   0%|          | 0/261 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [97]:
import pandas as pd

# Define the file paths in Google Drive using the determined date
three_month_file = f'/content/drive/MyDrive/Colab Notebooks/Correlation_Data/20250619_three_month_spearman_lagged_correlation.csv'
six_month_file = f'/content/drive/MyDrive/Colab Notebooks/Correlation_Data/20250619_six_month_spearman_lagged_correlation.csv'

# Check if files exist and load them

three_month_spearman_lagged_correlations = pd.read_csv(three_month_file, index_col=0)
print(f"Loaded existing file: {three_month_file}")
six_month_spearman_lagged_correlations = pd.read_csv(six_month_file, index_col=0)
print(f"Loaded existing file: {six_month_file}")

Loaded existing file: /content/drive/MyDrive/Colab Notebooks/Correlation_Data/20250619_three_month_spearman_lagged_correlation.csv
Loaded existing file: /content/drive/MyDrive/Colab Notebooks/Correlation_Data/20250619_six_month_spearman_lagged_correlation.csv


## Process correlated Data and get gravitational scores

In [98]:
import pandas as pd
import numpy as np

# Function to process stock correlation data, calculate gravitational forces,
# and filter connections for visualization based on the force.
def process_and_score_stocks(
    six_month_correlations,
    three_month_correlations,
    screener_data_df,
    source_ticker,
    min_nodes,
    max_nodes,
    threshold_percent
):
    """
    Processes stock correlation data for a specific source ticker.
    It filters for positive correlations, computes a dynamic impact score (gravitational_force),
    filters connections, and then calculates a final net gravitational force and the
    maximum potential force under ideal conditions.

    Args:
      six_month_correlations: The six-month spearman lagged correlation matrix.
      three_month_correlations: The three-month spearman lagged correlation matrix.
      screener_data_df: DataFrame with additional stock information.
      source_ticker: The ticker symbol for which to process data.
      min_nodes: Minimum number of correlated stocks to return.
      max_nodes: Maximum number of correlated stocks to return.
      threshold_percent: A percentage (0.0 to 1.0) of the maximum force to use as a filter.

    Returns:
      processed_data_df: A pandas DataFrame with processed data for visualization.
      source_data_df: A pandas DataFrame containing the net_gravitational_force,
                      max_potential_force, and gravitational_impact for the source ticker,
                      along with the source ticker's market cap influence and source_planet_radius.
    """
    # --- Data Unpivoting and Initial Setup ---
    # Start with the 6-month correlation data as the base
    correlation_df = six_month_correlations.rename_axis('source', axis=0)
    grouped_correlation_data = correlation_df.stack().reset_index()
    grouped_correlation_data.columns = ['source', 'target', 'six_month_spearman_correlation']

    grouped_correlation_data = grouped_correlation_data[
        (grouped_correlation_data['source'] != grouped_correlation_data['target']) &
        (grouped_correlation_data['target'] != source_ticker)
    ].copy()

    # --- Filter for the specific source ticker ---
    source_connections = grouped_correlation_data[grouped_correlation_data['source'] == source_ticker].copy()
    if source_connections.empty:
        print(f"No correlation data found for source ticker {source_ticker}.")
        # Return empty dataframes when no data is found
        return pd.DataFrame(), pd.DataFrame()

    # Add 3-month correlation data before filtering
    source_connections['three_month_spearman_correlation'] = source_connections.apply(
        lambda row: three_month_correlations.loc[row['source'], row['target']] if row['source'] in three_month_correlations.index and row['target'] in three_month_correlations.columns else 0, axis=1
    )

    # We only care about positively correlated stocks for this model in both 6 and 3 month periods
    positive_corr_group = source_connections[
        (source_connections['six_month_spearman_correlation'] > 0) &
        (source_connections['three_month_spearman_correlation'] > 0)
    ].copy()

    if positive_corr_group.empty:
        print(f"No positive correlations found for source ticker {source_ticker}.")
        # Return empty dataframes when no data is found
        return pd.DataFrame(), pd.DataFrame()

    # --- Enrich Data (before filtering) ---
    # Add market data
    screener_cols_to_add = ['code', 'market_capitalization', 'last_day_change']
    required_screener_cols = ['code', 'market_capitalization', 'last_day_change']
    if not all(col in screener_data_df.columns for col in required_screener_cols):
        missing = [col for col in required_screener_cols if col not in screener_data_df.columns]
        raise ValueError(f"screener_data_df is missing required columns: {missing}")

    screener_info = screener_data_df[screener_cols_to_add].rename(columns={'code': 'target'})
    positive_corr_group = pd.merge(positive_corr_group, screener_info, on='target', how='left')
    positive_corr_group.dropna(subset=['market_capitalization', 'last_day_change'], inplace=True)
    if positive_corr_group.empty:
        print(f"No valid connections after merging screener data for {source_ticker}.")
        # Return empty dataframes when no data is found
        return pd.DataFrame(), pd.DataFrame()


    # --- Calculate Dynamic Impact Score (Gravitational Force) ---
    epsilon = 1e-9 # Small value to avoid log(0) issues.
    # Weights for recency bias
    w_3m = 0.6
    w_6m = 0.4
    # "unified_correlation" is a weighted average of recent correlations.
    positive_corr_group['unified_correlation'] = (
        w_3m * positive_corr_group['three_month_spearman_correlation'] +
        w_6m * positive_corr_group['six_month_spearman_correlation']
    )

    # Calculate a market cap influence score scaled between 0 and 1 for target stocks.
    positive_corr_group['Market Cap'] = positive_corr_group['market_capitalization']

    # --- Calculate source ticker's market cap and log cap ---
    source_screener_info = screener_data_df[screener_data_df['code'] == source_ticker]
    source_market_cap = source_screener_info['market_capitalization'].iloc[0] if not source_screener_info.empty and 'market_capitalization' in source_screener_info.columns else epsilon
    source_log_cap = np.log(max(source_market_cap, epsilon))


    # Calculate log market caps for all relevant tickers (source and targets)
    all_market_caps = positive_corr_group['Market Cap'].tolist()
    all_market_caps.append(source_market_cap) # Include source market cap

    log_caps = np.log(pd.Series(all_market_caps).clip(lower=epsilon))

    min_log_cap, max_log_cap = log_caps.min(), log_caps.max()
    log_cap_range = max_log_cap - min_log_cap

    # Calculate market cap influence for target stocks
    if log_cap_range > 0:
        positive_corr_group['market_cap_influence'] = np.log(positive_corr_group['Market Cap'].clip(lower=epsilon))
    else:
        positive_corr_group['market_cap_influence'] = 20 # Neutral value if all caps are the same


    # The `gravitational_force` is a product of recent correlation strength and market influence.
    # Modified: Increased the influence of unified_correlation by multiplying by a factor
    correlation_weight_factor = 1.0 # Factor to increase the influence of unified_correlation
    positive_corr_group['gravitational_force'] = (
        (positive_corr_group['unified_correlation'] * correlation_weight_factor) * # Multiply unified_correlation by a factor
        positive_corr_group['market_cap_influence']
    )

    # --- Apply Filtering ---
    max_abs_force = positive_corr_group['gravitational_force'].abs().max()
    if pd.isna(max_abs_force) or max_abs_force == 0:
        # Return empty dataframes when no data is found
        return pd.DataFrame(), pd.DataFrame()

    force_threshold = max_abs_force * threshold_percent
    filtered_by_force_threshold = positive_corr_group[positive_corr_group['gravitational_force'].abs() >= force_threshold].copy()

    # Enforce min/max node constraints
    if len(filtered_by_force_threshold) < min_nodes:
        final_filtered_df = positive_corr_group.sort_values(by='gravitational_force', key=abs, ascending=False).head(min_nodes).copy()
    elif len(filtered_by_force_threshold) > max_nodes:
        final_filtered_df = filtered_by_force_threshold.sort_values(by='gravitational_force', key=abs, ascending=False).head(max_nodes).copy()
    else:
        final_filtered_df = filtered_by_force_threshold.copy()

    if final_filtered_df.empty:
        print(f"No connections remained for {source_ticker} after filtering.")
        # Return empty dataframes when no data is found
        return pd.DataFrame(), pd.DataFrame()

    # --- Calculate Final Net Force and Visualization Parameters ---
    final_filtered_df['Daily Change'] = final_filtered_df['last_day_change']

    final_filtered_df['signed_gravitational_force'] = final_filtered_df.apply(
        lambda row: row['gravitational_force'] if row['Daily Change'] >= 0 else -row['gravitational_force'],
        axis=1
    )

    net_gravitational_force = final_filtered_df['signed_gravitational_force'].sum()
    max_potential_force = final_filtered_df['market_cap_influence'].sum()

    # --- Calculate Visualization Parameters ---
    min_corr, max_corr = final_filtered_df['gravitational_force'].min(), final_filtered_df['gravitational_force'].max()
    corr_range = max_corr - min_corr if max_corr > min_corr else 1.0
    # MODIFIED: Reverse the scaling for Orbital Radius
    if corr_range > 0:
        final_filtered_df['Orbital Radius'] = 1 - ((final_filtered_df['gravitational_force'] - min_corr) / corr_range)
    else:
        final_filtered_df['Orbital Radius'] = 0.5 # Neutral value if all forces are the same

    # -----Calculate Planet Radius------
    # Combine all market caps to find the true min and max for normalization
    all_caps = pd.concat([
        final_filtered_df['Market Cap'],
        pd.Series([source_market_cap]) # Make sure source_market_cap is a Series
    ], ignore_index=True)

    # Calculate the log, clipping to avoid errors with zero
    log_all_caps = np.log(all_caps.clip(lower=epsilon))

    # Find the min and max from the complete set of data
    min_log_cap = log_all_caps.min()
    max_log_cap = log_all_caps.max()
    log_cap_range = max_log_cap - min_log_cap

    # Now, apply the normalization ONLY to the DataFrame's data
    # using the min/max from the combined set
    if log_cap_range > 0:
        # We are calculating log on just the dataframe column now
        log_df_caps = np.log(final_filtered_df['Market Cap'].clip(lower=epsilon))
        final_filtered_df['Planet Radius'] = (log_df_caps - min_log_cap) / log_cap_range
    else:
        # If all values are the same, assign a default radius
        final_filtered_df['Planet Radius'] = 0.5

    # Calculate source_planet_radius using the same min/max log caps from the targets and source.
    if log_cap_range > 0:
        source_planet_radius = (source_log_cap - min_log_cap) / log_cap_range
    else:
        source_planet_radius = 0.5 # Neutral value if all caps are the same

    # --- Final Cleanup and Column Selection ---
    # "gravitational_percent" shows the relative % contribution of each stock.
    final_filtered_df['gravitational_percent'] = (final_filtered_df['signed_gravitational_force'] / final_filtered_df['gravitational_force'].sum()) * 100

    final_columns = [
        'source', 'target', 'Daily Change', 'six_month_spearman_correlation',
        'three_month_spearman_correlation', 'unified_correlation',
        'Orbital Radius', 'Market Cap', 'Planet Radius', 'market_cap_influence',
        'gravitational_force', 'signed_gravitational_force', 'gravitational_percent'
    ]


    gravitational_impact = (net_gravitational_force / max_potential_force) * 100 if max_potential_force > 0 else 0

    # Use the same min_log_cap and log_cap_range from target stocks for scaling
    source_market_cap_influence = 20 if log_cap_range <= 0 else (source_log_cap)

    # Create source_data_df
    source_data_df = pd.DataFrame([{
        'ticker': source_ticker,
        'net_gravitational_force': net_gravitational_force,
        'max_potential_force': max_potential_force,
        'gravitational_impact': gravitational_impact,
        'source_market_cap_influence': source_market_cap_influence, # Add the source influence
        'source_planet_radius': source_planet_radius # Add the source planet radius
    }])


    for col in final_columns:
        if col not in final_filtered_df.columns:
            final_filtered_df[col] = np.nan

    processed_data_df = final_filtered_df[final_columns].copy()

    return processed_data_df, source_data_df


## ---------- MODIFIED: Run App ---------------
min_nodes = 5
max_nodes = 30
threshold_percent = 0.9

# User input Ticker
source_ticker = 'AAPL'

# Process the data for the network diagram
processed_data_df, source_data_df = process_and_score_stocks(
    six_month_spearman_lagged_correlations,
    three_month_spearman_lagged_correlations,
    screener_data_df,
    source_ticker,
    min_nodes,
    max_nodes,
    threshold_percent,
    )

# Extract the scalar values from the source_data_df for plotting
net_gravitational_force = source_data_df['net_gravitational_force'].iloc[0]
max_potential_force = source_data_df['max_potential_force'].iloc[0]
gravitational_impact = source_data_df['gravitational_impact'].iloc[0]
market_cap_influence = source_data_df['source_market_cap_influence'].iloc[0]
source_planet_radius = source_data_df['source_planet_radius'].iloc[0]

print(f"Net Gravitational Force: {net_gravitational_force:.2f}")
print(f"Max Potential Gravitational Force: {max_potential_force:.2f}")
print(f"Net Gravitaional Impact: {gravitational_impact:.2f}%")
print(f"Source Market Cap Influence: {market_cap_influence}")
print(f"Source Planet Radius: {source_planet_radius}")
print('----------------------------------')
processed_data_df

Net Gravitational Force: -51.83
Max Potential Gravitational Force: 120.00
Net Gravitaional Impact: -43.19%
Source Market Cap Influence: 28.7080963539423
Source Planet Radius: 1.0
----------------------------------


,source,target,Daily Change,six_month_spearman_correlation,three_month_spearman_correlation,unified_correlation,Orbital Radius,Market Cap,Planet Radius,market_cap_influence,gravitational_force,signed_gravitational_force,gravitational_percent
131,AAPL,TEAM,-3.370042,0.869194,0.762829,0.805375,0.000000,51487928320,0.269432,24.664613,19.864261,-19.864261,-23.629021
27,AAPL,COO,-0.487036,0.806244,0.620348,0.694706,0.892554,13928001536,0.033206,23.357167,16.226370,-16.226370,-19.301662
110,AAPL,PCAR,0.297619,0.798043,0.560664,0.655615,0.918810,47624282112,0.255339,24.586609,16.119357,16.119357,19.174367
37,AAPL,ENTG,-1.075551,0.834329,0.599474,0.693416,0.931216,11589699584,0.000000,23.173383,16.068790,-16.068790,-19.114216
55,AAPL,GEHC,-0.848519,0.789664,0.560140,0.651950,1.000000,32917352448,0.188607,24.217266,15.788440,-15.788440,-18.780734


##List of Top Predictions

In [99]:
import pandas as pd
top_gravitational_impacts = []

# Ensure unified_correlation_df is created from six_month_spearman_lagged_correlations
# assuming unified_correlation_df is meant to represent the base correlations
# based on the usage in the provided code for process_and_score_stocks.
# If unified_correlation_df should be different, please adjust this.
unified_correlation_df = six_month_spearman_lagged_correlations.copy()


# Iterate through each ticker in the unified_correlation_df
for ticker in tqdm(unified_correlation_df.index, desc="Processing tickers for gravitational impact"):
    try:
        # Run each ticker through the process_and_score_stocks function
        # Note: The function previously returned processed_data_df and prediction_score.
        # It now returns processed_data_df, net_gravitational_force, max_potential_force, gravitational_impact
        # We need to unpack the new return values.
        processed_df, source_data = process_and_score_stocks(
            six_month_spearman_lagged_correlations, # Pass the 6-month correlation
            three_month_spearman_lagged_correlations, # Pass the 3-month correlation
            screener_data_df,
            ticker, # Use the current ticker as source_ticker
            min_nodes,
            max_nodes,
            threshold_percent,
        )

        # Append the results to the list
        # Access the scalar values from the returned source_data DataFrame
        if not source_data.empty:
            top_gravitational_impacts.append({
                'ticker': ticker,
                'net_gravitational_force': source_data['net_gravitational_force'].iloc[0],
                'max_potential_force': source_data['max_potential_force'].iloc[0],
                'gravitational_impact': source_data['gravitational_impact'].iloc[0]
            })
    except Exception as e:
        print(f"Error processing ticker {ticker}: {e}")

# Create a DataFrame from the results
gravitational_impact_df = pd.DataFrame(top_gravitational_impacts)

# Sort by gravitational_impact in descending order for top positive impacts
top_positive_impacts = gravitational_impact_df.sort_values(by='gravitational_impact', ascending=False).head(10).reset_index(drop=True)

# Sort by gravitational_impact in ascending order for top negative impacts
top_negative_impacts = gravitational_impact_df.sort_values(by='gravitational_impact', ascending=True).head(10).reset_index(drop=True)


# Display the tables
print("Top 10 Positive Gravitational Impact Tickers:")
print(top_positive_impacts[['ticker', 'net_gravitational_force', 'max_potential_force', 'gravitational_impact']].to_string(index=False, formatters={'net_gravitational_force': '{:.2f}'.format, 'max_potential_force': '{:.2f}'.format, 'gravitational_impact': '{:.2f}'.format}))

print("\nTop 10 Negative Gravitational Impact Tickers:")
print(top_negative_impacts[['ticker', 'net_gravitational_force', 'max_potential_force', 'gravitational_impact']].to_string(index=False, formatters={'net_gravitational_force': '{:.2f}'.format, 'max_potential_force': '{:.2f}'.format, 'gravitational_impact': '{:.2f}'.format}))

Processing tickers for gravitational impact:   0%|          | 0/259 [00:00<?, ?it/s]

Top 10 Positive Gravitational Impact Tickers:
ticker net_gravitational_force max_potential_force gravitational_impact
  LINE                   88.21              120.27                73.34
  NTNX                  119.75              196.98                60.79
  CRWD                   67.66              131.53                51.44
  GLPI                   83.91              172.61                48.61
 FCNCA                   57.61              123.70                46.57
  CHRW                   53.36              121.18                44.03
   KHC                   52.24              122.29                42.72
   KDP                   52.80              125.98                41.91
   PEP                   48.94              117.39                41.69
   AEP                   47.82              120.01                39.85

Top 10 Negative Gravitational Impact Tickers:
ticker net_gravitational_force max_potential_force gravitational_impact
   WDC                 -124.60             

#Dash App with Solar System

##Setup

In [37]:
# Dash App Setup

#Terminal Command
#ngrok config add-authtoken 2yPWVwCXJDsY9JkhLhnrn549R5E_5b2uFuh7R4ER3fLJWLU59

# Install necessary libraries (run this cell first)
!pip install -q dash dash-bootstrap-components plotly pyngrok

# --- Import necessary libraries ---
from dash import Dash, html, dcc, Input, Output # We need dcc for interactive components and Input/Output for callbacks
import os
from pyngrok import ngrok
from google.colab import userdata

# # --- Configure Ngrok for public access ---
# NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
# os.environ["NGROK_AUTH_TOKEN"] = NGROK_AUTH_TOKEN

# --- Configure Ngrok for public access ---
# This part remains the same to create a public URL for your app
try:
    NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    # Use pyngrok to set the auth token programmatically
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
except userdata.SecretNotFoundError:
    print("NGROK_AUTH_TOKEN not found in Colab secrets. Please add it.")
    # You might want to handle this case, e.g., by exiting or using a default token if applicable
    exit()


# Define the port your Dash app will run on.
DASH_PORT = 8051

In [79]:
# Kills all ngrok processes that pyngrok started
print("Attempting to kill all active ngrok tunnels...")
ngrok.kill()
print("All ngrok tunnels should now be terminated.")

# Add a short delay to ensure the process is killed
import time
time.sleep(3)

Attempting to kill all active ngrok tunnels...
All ngrok tunnels should now be terminated.


In [80]:
# Create a public ngrok tunnel to the specified port.
print("Establishing Ngrok tunnel...")
public_url = ngrok.connect(DASH_PORT)
print(f"🚀 Your Dash app will be accessible at: {public_url}")

Establishing Ngrok tunnel...
🚀 Your Dash app will be accessible at: NgrokTunnel: "https://b031-35-229-160-204.ngrok-free.app" -> "http://localhost:8051"


## Dash App - Low Poly

In [103]:
# This is a dash app by Chris Parks
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.colors as mcolors
from dash import Dash, dcc, html, callback_context
from dash.dependencies import Input, Output, State, ALL
from dash.exceptions import PreventUpdate
import json
import base64

# --- Helper Functions ---

def create_model_image_svg(base_color, subdivisions, texture_spots_count):
    """
    Creates a base64 encoded SVG data URL that is a 2D representation of the 3D low-poly model.
    This function generates the geometry, projects it, and simulates flat shading to match the plot.
    """
    # 1. Generate the 3D model's vertices and faces for a unit sphere
    # We pass 0 for texture_spots here because we'll draw them separately in the SVG
    vertices, faces, _ = create_low_poly_sphere(0, 0, 0, 1, base_color, subdivisions, 0)

    # 2. Define a light source for shading the facets
    light_source = np.array([-0.5, 0.8, 1.0])
    light_source = light_source / np.linalg.norm(light_source)

    # 3. Process each face for rendering
    face_data = []
    for face in faces:
        # Get the vertices for the current face
        v0, v1, v2 = vertices[face]

        # --- Back-face culling: Don't render faces pointing away from the camera ---
        # The camera is at (0, 0, z), so we check the z-component of the normal
        normal = np.cross(v1 - v0, v2 - v0)
        if np.linalg.norm(normal) == 0: continue
        normal = normal / np.linalg.norm(normal)
        if normal[2] < 0:
            continue # This face is on the back of the sphere, so we skip it

        # --- Shading: Calculate brightness based on angle to the light source ---
        intensity = np.dot(normal, light_source)
        # Map intensity to a brightness factor for the color
        color_factor = 0.65 + intensity * 0.5
        facet_color = darken_color(base_color, color_factor)

        # --- Projection: Convert 3D vertex coordinates to 2D SVG coordinates ---
        # We scale and shift the (x, y) coordinates to fit in a 100x100 SVG
        points_2d_str = " ".join([f"{(v[0] * 48) + 50},{(v[1] * -48) + 50}" for v in [v0, v1, v2]])

        # Store the face's z-depth for sorting, so closer faces draw on top
        avg_z = (v0[2] + v1[2] + v2[2]) / 3
        face_data.append({'z': avg_z, 'points': points_2d_str, 'color': facet_color})

    # 4. Sort faces from back to front
    face_data.sort(key=lambda f: f['z'])

    # 5. Build the SVG polygons from the sorted face data
    svg_polygons = "".join(f'<polygon points="{f["points"]}" fill="{f["color"]}" />' for f in face_data)

    # 6. Add texture spots as random circles
    texture_color = darken_color(base_color, 0.7)
    svg_texture_spots = ""
    np.random.seed(sum(ord(c) for c in base_color)) # Seed for consistency
    for _ in range(texture_spots_count):
        angle = np.random.uniform(0, 2 * np.pi)
        radius = np.random.uniform(0, 48)
        spot_size = np.random.uniform(4, 9)
        cx = 50 + radius * np.cos(angle)
        cy = 50 + radius * np.sin(angle)
        svg_texture_spots += f'<circle cx="{cx}" cy="{cy}" r="{spot_size}" fill="{texture_color}" opacity="0.7"/>'

    # 7. Assemble the final SVG string
    svg_string = f'''
    <svg width="100" height="100" viewBox="0 0 100 100" xmlns="http://www.w3.org/2000/svg">
      <defs>
        <clipPath id="sphereClip">
          <circle cx="50" cy="50" r="48"/>
        </clipPath>
        <filter id="blur-effect">
          <feGaussianBlur in="SourceGraphic" stdDeviation="0.7" />
        </filter>
      </defs>
      <g clip-path="url(#sphereClip)" filter="url(#blur-effect)">
        {svg_polygons}
        {svg_texture_spots}
      </g>
      <circle cx="50" cy="50" r="48" fill="none" stroke="rgba(255, 255, 255, 0.25)" stroke-width="1.5" />
    </svg>
    '''
    encoded_svg = base64.b64encode(svg_string.encode('utf-8')).decode('utf-8')
    return f"data:image/svg+xml;base64,{encoded_svg}"

def darken_color(color_hex, factor=0.8):
    """Darkens or lightens a hex color by a given factor."""
    rgb = mcolors.to_rgb(color_hex)
    # Clamp values to ensure they stay within the valid [0, 1] range for RGB
    modified_rgb = [min(max(c * factor, 0), 1) for c in rgb]
    return mcolors.to_hex(modified_rgb)

def create_low_poly_sphere(center_x, center_y, center_z, radius, base_color, subdivisions=2, texture_spots=15):
    """
    Generates vertex and face data for a textured low-poly sphere (icosphere).
    """
    # Define the 12 vertices of a regular icosahedron
    t = (1.0 + np.sqrt(5.0)) / 2.0
    vertices = np.array([
        [-1,  t,  0], [ 1,  t,  0], [-1, -t,  0], [ 1, -t,  0],
        [ 0, -1,  t], [ 0,  1,  t], [ 0, -1, -t], [ 0,  1, -t],
        [ t,  0, -1], [ t,  0,  1], [-t,  0, -1], [-t,  0,  1]
    ])

    # Define the 20 triangular faces of the icosahedron
    faces = np.array([
        [0, 11, 5], [0, 5, 1], [0, 1, 7], [0, 7, 10], [0, 10, 11],
        [1, 5, 9], [5, 11, 4], [11, 10, 2], [10, 7, 6], [7, 1, 8],
        [3, 9, 4], [3, 4, 2], [3, 2, 6], [3, 6, 8], [3, 8, 9],
        [4, 9, 5], [2, 4, 11], [6, 2, 10], [8, 6, 7], [9, 8, 1]
    ])

    # Subdivide faces to create more polygons
    for _ in range(subdivisions):
        new_faces = []
        mid_points = {}
        for face in faces:
            v_indices = [face[0], face[1], face[2]]
            new_v_indices = []
            for i in range(3):
                v1 = v_indices[i]
                v2 = v_indices[(i + 1) % 3]
                mid_key = tuple(sorted((v1, v2)))
                mid_idx = mid_points.get(mid_key)
                if mid_idx is None:
                    mid_idx = len(vertices)
                    vertices = np.vstack([vertices, (vertices[v1] + vertices[v2]) / 2.0])
                    mid_points[mid_key] = mid_idx
                new_v_indices.append(mid_idx)
            new_faces.append([v_indices[0], new_v_indices[0], new_v_indices[2]])
            new_faces.append([v_indices[1], new_v_indices[1], new_v_indices[0]])
            new_faces.append([v_indices[2], new_v_indices[2], new_v_indices[1]])
            new_faces.append(new_v_indices)
        faces = np.array(new_faces)

    # Normalize vertices to form a sphere, then scale and translate
    vertices = vertices / np.linalg.norm(vertices, axis=1)[:, np.newaxis]
    final_vertices = vertices * radius + np.array([center_x, center_y, center_z])

    # Create vertex colors for texture spots
    darker_color_hex = darken_color(base_color, 0.7)
    vertex_colors = [base_color] * len(final_vertices)
    if texture_spots > 0:
        spot_indices = np.random.choice(len(final_vertices), size=texture_spots, replace=False)
        for idx in spot_indices:
            vertex_colors[idx] = darker_color_hex

    return final_vertices, faces, vertex_colors

# --- Define New Color Palette and Colormap ---
# Define the red and green color spectrums
RED_SPECTRUM = {'light': '#ee868f', 'dark': '#cd2d35'}
GREEN_SPECTRUM = {'light': '#8ac08b', 'dark': '#329542'}

# Create separate colormaps for negative (red) and positive (green) values
red_cmap = mcolors.LinearSegmentedColormap.from_list('red_cmap', [RED_SPECTRUM['light'], RED_SPECTRUM['dark']])
green_cmap = mcolors.LinearSegmentedColormap.from_list('green_cmap', [GREEN_SPECTRUM['light'], GREEN_SPECTRUM['dark']])

def get_node_color(value, min_val, max_val):
    """
    Determines the node color based on its value, using a diverging red/green scale.
    """
    if value >= 0:
        if value >= max_val:
            return GREEN_SPECTRUM['dark']
        norm_val = value / max_val if max_val != 0 else 0
        return mcolors.to_hex(green_cmap(norm_val))
    else: # value < 0
        if value <= min_val:
            return RED_SPECTRUM['dark']
        norm_val = value / min_val if min_val != 0 else 0
        return mcolors.to_hex(red_cmap(norm_val))


# --- Main Plotting Function ---
def solar_system_visual(source_ticker, processed_data_df, source_data_df, screener_data_df):
    """
    Creates the main 3D solar system visualization with a low-poly aesthetic.
    """
    ticker_connections = processed_data_df[processed_data_df['source'] == source_ticker].copy()
    source_info_row = source_data_df[source_data_df['ticker'] == source_ticker]

    if ticker_connections.empty or source_info_row.empty:
        return go.Figure().update_layout(title=f"Data not available for {source_ticker}", title_x=0.5, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font_color='white')

    source_info = source_info_row.iloc[0]
    fig = go.Figure()

    pos = {source_ticker: (0, 0, 0)}
    actual_target_connections = ticker_connections[ticker_connections['target'] != source_ticker].copy()
    num_connections = len(actual_target_connections)
    radii_for_rings = []

    if num_connections > 0:
        original_radii = actual_target_connections['Orbital Radius']
        min_rad, max_rad = original_radii.min(), original_radii.max()
        rad_range = max_rad - min_rad if max_rad > min_rad else 1.0
        min_visual_radius, max_visual_radius = 3.0, 10.0
        visual_range = max_visual_radius - min_visual_radius
        thetas = np.linspace(0, 2 * np.pi, num_connections, endpoint=False)

        for i, (index, row) in enumerate(actual_target_connections.iterrows()):
            scaled_radius = ((row['Orbital Radius'] - min_rad) / rad_range) * visual_range + min_visual_radius
            radii_for_rings.append(scaled_radius)
            theta = thetas[i]
            pos[row['target']] = (scaled_radius * np.cos(theta), scaled_radius * np.sin(theta), 0)

    # --- Add Orbital Rings ---
    for r in sorted(list(set(radii_for_rings))):
        theta_ring = np.linspace(0, 2 * np.pi, 100)
        fig.add_trace(go.Scatter3d(
            x=r * np.cos(theta_ring), y=r * np.sin(theta_ring), z=np.zeros(100),
            mode='lines',
            line=dict(color='rgba(255, 255, 255, 0.2)', width=1, dash='solid'),
            hoverinfo='none'
        ))

    scene_annotations = []

    # --- Loop Through Each Node to Draw Them ---
    for node_name, coords in pos.items():
        center_x, center_y, center_z = coords
        is_source = (node_name == source_ticker)

        screener_info_row = screener_data_df[screener_data_df['code'] == node_name]
        if screener_info_row.empty: continue
        screener_info = screener_info_row.iloc[0]

        market_cap = screener_info.get('market_capitalization', 0)
        market_cap_str = f"${market_cap/1e12:.2f}T" if market_cap > 1e12 else f"${market_cap/1e9:.2f}B"
        min_visual_size, max_visual_size = 0.6, 1.5

        if is_source:
            hover_text = (f"<b>{screener_info.get('name', node_name)} ({node_name})</b><br>"
                          f"Industry: {screener_info.get('industry', 'N/A')}<br>"
                          f"Sector: {screener_info.get('sector', 'N/A')}<br>"
                          f"Avg Volume (1d): {screener_info.get('avgvol_1d', 'N/A')}<br>"
                          f"Market Cap: {market_cap_str}")
            node_color = get_node_color(source_info['gravitational_impact'], -80, 80)
            radius = min_visual_size + (source_info['source_planet_radius'] * (max_visual_size - min_visual_size))
            subdivisions = 2
        else:
            processed_info = ticker_connections[ticker_connections['target'] == node_name].iloc[0]
            hover_text = (f"<b>{screener_info.get('name', node_name)} ({node_name})</b><br>"
                          f"Industry: {screener_info.get('industry', 'N/A')}<br>"
                          f"Sector: {screener_info.get('sector', 'N/A')}<br>"
                          f"Avg Volume (1d): {screener_info.get('avgvol_1d', 'N/A')}<br>"
                          f"Daily Change: {processed_info['Daily Change']:.2f}%<br>"
                          f"Market Cap: {market_cap_str}")
            node_color = get_node_color(processed_info['Daily Change'], -5, 5)
            radius = min_visual_size + (processed_info['Planet Radius'] * (max_visual_size - min_visual_size))
            subdivisions = 2

        vertices, faces, vertex_colors = create_low_poly_sphere(center_x, center_y, center_z, radius, node_color, subdivisions, texture_spots=15)
        fig.add_trace(go.Mesh3d(
            x=vertices[:, 0], y=vertices[:, 1], z=vertices[:, 2],
            i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
            vertexcolor=vertex_colors,
            opacity=1.0,
            flatshading=True,
            # Add lighting properties to make edges crisp
            lighting=dict(
                ambient=0.8,
                diffuse=0.5,
                specular=1,
                roughness=1
            ),
            hoverinfo='text',
            text=hover_text,
            hoverlabel=dict(bgcolor='#0f0524', font=dict(color='#EAEAEA', size=14), bordercolor='rgba(255, 255, 255, 0.3)')
        ))

        # Add labels as scene annotations
        scene_annotations.append(
            dict(
                x=center_x,
                y=center_y,
                z=center_z, # Position label in the center of the sphere
                text=f"<b>{node_name}</b>",
                showarrow=False,
                font=dict(color='white', size=14),
                bgcolor="rgba(0,0,0,0)",
                xanchor="center"
            )
        )


    # --- Configure Final Layout ---
    fig.update_layout(
        scene=dict(
            xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False),
            camera=dict(eye=dict(x=0.9, y=0.9, z=0.9)),
            aspectmode='data',
            annotations=scene_annotations,
            bgcolor='rgba(0,0,0,0)'
        ),
        margin=dict(l=0, r=0, b=0, t=0),
        showlegend=False,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        font=dict(family="'Space Grotesk', sans-serif", color='#EAEAEA', size=16)
    )
    return fig

# --- Dash App Definition & Styling ---
app = Dash(__name__, external_stylesheets=['https://fonts.googleapis.com/css2?family=Space+Grotesk:wght@400;700&display=swap'])
server = app.server

# --- New Theme Styles ---
THEME = {
    'background': '#0B041A',
    'text': '#EAEAEA',
    'primary': '#FFFFFF',
    'container_bg': 'rgba(30, 15, 60, 0.3)',
    'container_border': 'rgba(255, 255, 255, 0.1)'
}

# Starry background style
starry_background_style = {
    'backgroundColor': THEME['background'],
    'backgroundImage': 'radial-gradient(circle, white 0.5px, transparent 1.5px), radial-gradient(circle, white 1px, transparent 2px), radial-gradient(circle, white 0.5px, transparent 1.5px)',
    'backgroundSize': '350px 350px, 250px 250px, 150px 150px',
    'backgroundPosition': '0 0, 40px 60px, 130px 270px',
    'color': THEME['text'],
    'fontFamily': "'Space Grotesk', sans-serif",
    'minHeight': '100vh',
    'padding': '20px'
}

font_style = {'fontFamily': "'Space Grotesk', sans-serif", 'color': THEME['text']}
container_style = {
    'backgroundColor': THEME['container_bg'],
    'border': f"1px solid {THEME['container_border']}",
    'padding': '25px 30px',
    'borderRadius': '12px',
    'backdropFilter': 'blur(10px)',
    'width': '100%',
    'boxSizing': 'border-box'
}
header_style = {**font_style, 'color': THEME['text'], 'textAlign': 'center', 'fontWeight': 'bold', 'marginTop': 0, 'marginBottom': '20px', 'fontSize': '22px'}

# Assume dataframes are loaded
ticker_options = [{'label': row['name'] + f" ({row['code']})", 'value': row['code']} for index, row in screener_data_df.iterrows()]
default_ticker = 'AAPL'

# --- App Layout ---
app.layout = html.Div(style=starry_background_style, children=[
    html.H1("THE FINANCIAL PLANETARIUM", style={**font_style, 'textAlign': 'center', 'padding': '20px 0', 'fontSize': '32px', 'fontWeight': 'bold', 'letterSpacing': '4px'}),

    html.Div([
        dcc.Dropdown(id='ticker-dropdown', options=ticker_options, value=default_ticker, clearable=False, style={'height': '40px', 'color': 'black'})
    ], style={'width': '90%', 'maxWidth': '500px', 'margin': '0 auto 20px auto', 'backgroundColor': THEME['container_bg'], 'border': f"1px solid {THEME['container_border']}", 'borderRadius': '12px', 'backdropFilter': 'blur(10px)', 'padding': '5px'}),

    html.P(id='prediction-summary-text', style={'textAlign': 'center', 'padding': '10px 0', 'fontSize': '18px', 'color': THEME['text']}),
    html.Div(id='graph-container', style={'height': '65vh', 'width': '98%', 'margin': 'auto', 'borderRadius': '15px', 'boxShadow': '0 0 25px 5px rgba(255, 255, 255, 0.15)'}),
    html.Div(id='info-panels-container')
])


# --- Main Callback to Generate Content ---
@app.callback(
    [Output('graph-container', 'children'),
     Output('info-panels-container', 'children'),
     Output('prediction-summary-text', 'children')],
    Input('ticker-dropdown', 'value')
)
def update_layout_and_data(selected_ticker):

    global processed_data_df, source_data_df
    processed_data_df, source_data_df = process_and_score_stocks(
        six_month_spearman_lagged_correlations, three_month_spearman_lagged_correlations, screener_data_df,
        selected_ticker, min_nodes, max_nodes, threshold_percent
    )

    if processed_data_df.empty or source_data_df.empty:
        empty_fig = go.Figure().update_layout(title=f"Data not available for {selected_ticker}", title_x=0.5, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font_color='white')
        return dcc.Graph(id='network-graph', figure=empty_fig, style={'height': '100%'}), html.Div(f"Data not available for {selected_ticker}", style={'textAlign': 'center', 'padding': '20px'}), ""

    graph = dcc.Graph(id='network-graph', figure=solar_system_visual(selected_ticker, processed_data_df, source_data_df, screener_data_df), style={'height': '100%'})

    # --- Generate data for panels and summary text ---
    star_info_screener = screener_data_df[screener_data_df['code'] == selected_ticker].iloc[0]
    star_info_source = source_data_df[source_data_df['ticker'] == selected_ticker].iloc[0]
    grav_impact = star_info_source.get('gravitational_impact', 0)

    # Summary text
    source_name = star_info_screener.get('name', selected_ticker)
    direction = "increase" if grav_impact >= 0 else "decrease"
    strength = grav_impact
    prediction_summary = f"{source_name} ({selected_ticker}) is predicted to {direction} today with a prediction strength of {strength:.2f}%."

    # Star Info Panel
    star_color = get_node_color(grav_impact, -80, 80)
    star_image_src = create_model_image_svg(star_color, subdivisions=2, texture_spots_count=10)
    star_text_content = html.Div([
        html.P(f"Name: {star_info_screener['name']} ({selected_ticker})", style={'margin': '0 0 5px 0'}),
        html.P(f"Market Cap: ${star_info_screener.get('market_capitalization', 0)/1e12:.2f}T", style={'margin': '0 0 5px 0'}),
        html.P(f"Gravitational Impact: {grav_impact:.2f}%", style={'margin': '0'}),
    ])
    star_info_panel = html.Div([
        html.H3("Star Information", style=header_style),
        html.Div([
            html.Img(src=star_image_src, style={'height': '150px', 'width': '150px', 'marginRight': '20px'}),
            star_text_content
        ], style={'display': 'flex', 'alignItems': 'center'})
    ], style=container_style)

    # --- Predictions Panel ---
    prediction_items = []
    if 'top_positive_impacts' in globals() and not top_positive_impacts.empty and 'top_negative_impacts' in globals() and not top_negative_impacts.empty:
        combined_impacts = pd.concat([top_positive_impacts.head(5), top_negative_impacts.head(5)])
        prediction_items = [
            html.Div([
                html.Span(row['ticker']),
                html.Span(f"{row['gravitational_impact']:.2f}%", style={'color': '#4ade80' if row['gravitational_impact'] > 0 else '#f87171', 'fontWeight': 'bold'})
            ], style={'display': 'flex', 'justifyContent': 'space-between', 'padding': '8px 0', 'borderBottom': f'1px solid {THEME["container_border"]}'})
            for _, row in combined_impacts.iterrows()
        ]
    else:
         prediction_items = [html.Div("Top predictions not available.")]
    predictions_panel = html.Div([html.H3("Top Predictions", style=header_style), html.Div(prediction_items)], style=container_style)

    # --- Legend Panel ---
    legend_gradient_style = {'height': '15px', 'width': '100%', 'maxWidth': '300px', 'borderRadius': '5px', 'marginBottom': '8px', 'background': f"linear-gradient(to right, {RED_SPECTRUM['dark']}, {RED_SPECTRUM['light']} 49.9%, {GREEN_SPECTRUM['light']} 50.1%, {GREEN_SPECTRUM['dark']})"}
    legend_content = html.Div([
        html.Div(style=legend_gradient_style),
        html.Div([html.Span("Decrease"), html.Span("Increase")], style={'display': 'flex', 'justifyContent': 'space-between', 'fontSize': '12px', 'width': '100%', 'maxWidth': '300px'})
    ], style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center'})
    legend_panel_container = html.Div([
        html.H3("Legend", style={**header_style, 'marginBottom': '15px', 'textAlign': 'center'}),
        legend_content
    ], style={**container_style, 'padding': '15px 20px', 'width': '100%', 'align-self': 'center'})

    # --- Planet Info Table ---
    planets_df = processed_data_df[processed_data_df['source'] == selected_ticker].copy()
    table_header = [html.Thead(html.Tr([html.Th(col, style={'padding': '12px', 'textAlign': 'left', 'borderBottom': f"2px solid {THEME['container_border']}"}) for col in ["Ticker", "Name", "Market Cap", "Daily Change", "Correlation", "Grav. Force"]]))]
    table_rows = []
    if not planets_df.empty:
        for _, p_row in planets_df.iterrows():
            s_info = screener_data_df[screener_data_df['code'] == p_row['target']].iloc[0]
            planet_color = get_node_color(p_row['Daily Change'], -5, 5)
            planet_image_src = create_model_image_svg(planet_color, subdivisions=2, texture_spots_count=5)

            ticker_cell = html.Div([
                html.Img(src=planet_image_src, style={'height': '40px', 'width': '40px', 'marginRight': '10px'}),
                html.Span(p_row['target'])
            ], style={'display': 'flex', 'alignItems': 'center'})

            table_rows.append(html.Tr([
                html.Td(ticker_cell, style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(s_info['name'], style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(f"${s_info['market_capitalization']/1e9:.2f}B", style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(f"{p_row['Daily Change']:.2f}%", style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(f"{p_row['unified_correlation']:.3f}", style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(f"{p_row['signed_gravitational_force']:.2f}", style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
            ]))

    # Dynamically set the height of the table container
    table_wrapper_style = {'overflowX': 'auto'}
    if len(planets_df) > 5:
        table_wrapper_style['maxHeight'] = '300px'
        table_wrapper_style['overflowY'] = 'auto'

    planet_table_panel = html.Div([
        html.H3("Planet Information", style=header_style),
        html.Div(html.Table(table_header + [html.Tbody(table_rows)], style={'width': '100%', 'borderCollapse': 'collapse'}), style=table_wrapper_style)
    ], style=container_style)

    # Assemble the info panels layout
    info_panels = html.Div([
        legend_panel_container,
        star_info_panel,
        planet_table_panel,
        predictions_panel
    ], style={'display': 'flex', 'flexDirection': 'column', 'gap': '20px', 'padding': '20px 0'})

    return graph, info_panels, prediction_summary

# --- Run the App ---
if __name__ == '__main__':
    app.run(debug=True, port=8051)


<IPython.core.display.Javascript object>

#Draft Apps

##Terminal Aesthetic

In [ ]:
# The Financial Planetarium
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical operations, especially for generating coordinates
import plotly.graph_objects as go  # The main library for creating interactive plots
import matplotlib.colors as mcolors # For creating custom color maps and normalizing color scales
from dash import Dash, dcc, html, callback_context   # The framework for building the web application interface
from dash.dependencies import Input, Output, State, ALL # To create interactivity in the Dash app (callbacks)
from dash.exceptions import PreventUpdate
import json


# --- Helper Function to Create Wireframe Spheres ---
def create_wireframe_sphere(center_x, center_y, center_z, radius, color, n_lat=20, n_lon=20):
    """
    Generates the coordinate data for a 3D wireframe sphere for a Plotly graph.
    This function calculates points along latitude and longitude lines to give the
    appearance of a sphere without rendering a solid surface.

    Args:
        center_x, center_y, center_z (float): The center coordinates of the sphere.
        radius (float): The radius of the sphere.
        color (str): The color of the wireframe lines (in a format Plotly understands, e.g., hex code).
        n_lat (int): The number of latitude lines to draw. More lines create a denser sphere.
        n_lon (int): The number of longitude lines to draw. More lines create a denser sphere.

    Returns:
        tuple: A tuple containing three lists (x_lines, y_lines, z_lines) of coordinates
               for the sphere's lines. `None` values are used to separate the lines.
    """
    # Initialize lists to hold the coordinates for all line segments
    x_lines, y_lines, z_lines = [], [], []

    # --- Generate latitude lines (horizontal rings) ---
    # Loop through each latitude angle from the North Pole to the South Pole
    for i in range(n_lat + 1):
        # Calculate the latitude angle. np.pi/2 is the North Pole, -np.pi/2 is the South Pole.
        lat_angle = np.pi / 2 - (i * np.pi / n_lat)
        z = radius * np.sin(lat_angle) # Z coordinate is constant for a given latitude
        r = radius * np.cos(lat_angle) # The radius of the horizontal ring at this latitude

        # Loop through each longitude to draw the points on the ring
        for j in range(n_lon + 1):
            lon_angle = j * 2 * np.pi / n_lon # Go a full circle (2*pi)
            x = r * np.cos(lon_angle) # Calculate x based on the ring radius and longitude
            y = r * np.sin(lon_angle) # Calculate y based on the ring radius and longitude

            # Append the calculated point, shifted by the sphere's center coordinates
            x_lines.append(x + center_x)
            y_lines.append(y + center_y)
            z_lines.append(z + center_z)

        # Add `None` after each full latitude line. This tells Plotly to stop drawing
        # the current line and start a new one, preventing lines from connecting across the sphere.
        x_lines.append(None)
        y_lines.append(None)
        z_lines.append(None)

    # --- Generate longitude lines (vertical meridians) ---
    # Loop through each longitude angle
    for i in range(n_lon + 1):
        lon_angle = i * 2 * np.pi / n_lon # The angle for this specific meridian

        # Loop through each latitude to draw the points along the vertical line
        for j in range(n_lat + 1):
            lat_angle = np.pi / 2 - (j * np.pi / n_lat) # Go from North to South Pole
            z = radius * np.sin(lat_angle) # Calculate z for this point on the meridian
            r = radius * np.cos(lat_angle) # Calculate the horizontal distance from the z-axis

            # Calculate x and y based on the constant longitude angle
            x = r * np.cos(lon_angle)
            y = r * np.sin(lon_angle)

            # Append the calculated point, shifted by the sphere's center coordinates
            x_lines.append(x + center_x)
            y_lines.append(y + center_y)
            z_lines.append(z + center_z)

        # Add `None` after each full longitude line to start a new line.
        x_lines.append(None)
        y_lines.append(None)
        z_lines.append(None)

    return x_lines, y_lines, z_lines

# --- Define Color Mapping (Global for use in layout and plotting) ---
colors = ['#FF0000', '#F04435', '#E47C61', '#D9B28B', '#A5BA6F', '#73C255', '#33CC33']
cmap = mcolors.LinearSegmentedColormap.from_list('my_cmap', colors)

# --- Main Plotting Function ---
def solar_system_visual(source_ticker, processed_data_df, source_data_df, screener_data_df):
    """
    Creates the main 3D solar system visualization for a given central stock (source_ticker).

    Args:
        source_ticker (str): The ticker symbol for the central star of the system.
        processed_data_df (pd.DataFrame): DataFrame with data about connections (planets).
        source_data_df (pd.DataFrame): DataFrame with data about the source stocks (stars).
        screener_data_df (pd.DataFrame): DataFrame with general financial info for all tickers.

    Returns:
        go.Figure: A Plotly Figure object containing the complete 3D visualization.
    """
    # Filter the data to get only the connections for the selected source_ticker
    ticker_connections = processed_data_df[processed_data_df['source'] == source_ticker].copy()

    # Get the specific row of data for the source_ticker itself
    source_info_row = source_data_df[source_data_df['ticker'] == source_ticker]

    # If there's no data for the selected ticker, return an empty figure with a message
    if ticker_connections.empty or source_info_row.empty:
        return go.Figure().update_layout(title=f"Data not available for {source_ticker}", title_x=0.5, paper_bgcolor='black', font_color='white')

    # Extract the source info into a pandas Series for easier access
    source_info = source_info_row.iloc[0]

    # Initialize the Plotly Figure
    fig = go.Figure()

    # Create a dictionary to store the 3D positions (x, y, z) of each stock (node).
    # The source ticker (the "star") is always placed at the origin (0, 0, 0).
    pos = {source_ticker: (0, 0, 0)}

    # Filter out any connections where the target is the source itself
    actual_target_connections = ticker_connections[ticker_connections['target'] != source_ticker].copy()
    num_connections = len(actual_target_connections)

    # List to store the calculated visual radii for the orbital rings
    radii_for_rings = []

    # Calculate positions for the "planets" (connected tickers) if any exist
    if num_connections > 0:
        # --- Scale orbital distances for better visualization ---
        # Get the original orbital radii from the data
        original_radii = actual_target_connections['Orbital Radius']
        min_rad, max_rad = original_radii.min(), original_radii.max()

        # Avoid division by zero if all radii are the same
        rad_range = max_rad - min_rad if max_rad > min_rad else 1.0

        # Define the range for the visual radii in the plot (e.g., from 2 to 8 units)
        min_visual_radius, max_visual_radius = 2, 8.0
        visual_range = max_visual_radius - min_visual_radius

        # Generate evenly spaced angles to position the planets around the star
        thetas = np.linspace(0, 2 * np.pi, num_connections, endpoint=False)

        # Loop through each connected "planet" to calculate its position
        for i, (index, row) in enumerate(actual_target_connections.iterrows()):
            # Scale the original radius to the new visual range
            scaled_radius = ((row['Orbital Radius'] - min_rad) / rad_range) * visual_range + min_visual_radius
            radii_for_rings.append(scaled_radius)

            # Get the angle for this planet
            theta = thetas[i]

            # Calculate and store the (x, y, z) position using polar coordinates. Z is 0 for a flat system.
            pos[row['target']] = (scaled_radius * np.cos(theta), scaled_radius * np.sin(theta), 0)

    # --- Add Orbital Rings ---
    # Draw a dotted circle for each unique orbital radius
    for r in sorted(list(set(radii_for_rings))):
        theta_ring = np.linspace(0, 2 * np.pi, 100) # 100 points for a smooth circle
        fig.add_trace(go.Scatter3d(
            x=r * np.cos(theta_ring), y=r * np.sin(theta_ring), z=np.zeros(100),
            mode='lines',
            line=dict(color='rgba(100, 100, 100, 0.5)', width=1, dash='dot'), # Faint, dotted line
            hoverinfo='none' # These rings should not be interactive
        ))

    # Create normalizers to map financial data values to the 0-1 range for the colormap
    norm_daily_change = mcolors.Normalize(vmin=-5, vmax=5) # For planet colors
    norm_gravitational_impact = mcolors.Normalize(vmin=-100, vmax=100) # For star color

    # List to hold annotations (text labels) for the scene
    scene_annotations = []

    # --- Loop Through Each Node (Star and Planets) to Draw Them ---
    for node_name, coords in pos.items():
        center_x, center_y, center_z = coords
        is_source = (node_name == source_ticker)

        # Get financial info for the current node from the screener dataframe
        screener_info_row = screener_data_df[screener_data_df['code'] == node_name]
        if screener_info_row.empty: continue # Skip if no data is found
        screener_info = screener_info_row.iloc[0]

        # Format the market cap for readability (Billions or Trillions)
        market_cap = screener_info.get('market_capitalization', 0)
        market_cap_str = f"${market_cap/1e12:.2f}T" if market_cap > 1e12 else f"${market_cap/1e9:.2f}B"


        # Define the range for the visual size of the spheres
        min_visual_size, max_visual_size = 0.3, 1.0

        # --- Configure Node Properties (Text, Color, Size) ---
        if is_source: # If the node is the central star
            text = (f"<b>{screener_info.get('name', node_name)} ({node_name})</b><br>"
                    f"Industry: {screener_info.get('industry', 'N/A')}<br>"
                    f"Sector: {screener_info.get('sector', 'N/A')}<br>"
                    f"Avg Volume (1d): {screener_info.get('avgvol_1d', 'N/A')}<br>"
                    f"Market Cap: {market_cap_str}")
            # Color is based on gravitational impact
            node_color = mcolors.to_hex(cmap(norm_gravitational_impact(source_info['gravitational_impact'])))
            # Radius is based on its own calculated radius property, scaled to the visual size range
            radius = min_visual_size + (source_info['source_planet_radius'] * (max_visual_size - min_visual_size))

        else: # If the node is a planet
            processed_info_row = ticker_connections[ticker_connections['target'] == node_name]
            if processed_info_row.empty: continue # Skip if no processed data
            processed_info = processed_info_row.iloc[0]

            text = (f"<b>{screener_info.get('name', node_name)} ({node_name})</b><br>"
                    f"Industry: {screener_info.get('industry', 'N/A')}<br>"
                    f"Sector: {screener_info.get('sector', 'N/A')}<br>"
                    f"Avg Volume (1d): {screener_info.get('avgvol_1d', 'N/A')}<br>"
                    f"Daily Change: {processed_info['Daily Change']:.2f}%<br>"
                    f"Market Cap: {market_cap_str}")
            # Color is based on daily price change
            node_color = mcolors.to_hex(cmap(norm_daily_change(processed_info['Daily Change'])))
            # Radius is based on its own calculated radius property, scaled to the visual size range
            radius = min_visual_size + (processed_info['Planet Radius'] * (max_visual_size - min_visual_size))

        # --- Create and add the wireframe sphere for the current node ---
        sphere_x, sphere_y, sphere_z = create_wireframe_sphere(center_x, center_y, center_z, radius, node_color)
        fig.add_trace(go.Scatter3d(
            x=sphere_x, y=sphere_y, z=sphere_z,
            mode='lines',
            line=dict(color=node_color, width=2),
            hoverinfo='none' # The wireframe itself doesn't have a hover label
        ))

        # --- Dynamic Label Positioning using Scene Annotations ---
        # Add a text label for each node that is rendered on top of the 3D scene.
        scene_annotations.append(
            dict(
                x=center_x, y=center_y, z=center_z, # Position the label
                text=f"<b>{node_name}</b>",
                showarrow=False,
                # For cyan on a transparent background.
                font=dict(color='white' if not is_source else '#FFFFFF', size=17),
                # For planets, set the background color to the planet's color. For the star, it's transparent.
                bgcolor="rgba(0,0,0,0)"
            )
        )

        # --- HOVER INFO TRICK ---
        # Scene annotations don't support rich hover text. So, we add an INVISIBLE marker at the
        # center of each sphere. We make this marker large enough to be easily hovered over and
        # assign the detailed text to its `hovertext` property.
        fig.add_trace(go.Scatter3d(
            x=[center_x], y=[center_y], z=[center_z],
            mode='markers',
            marker=dict(size=radius*20, color='rgba(0,0,0,0)'), # Invisible, but sized for hovering
            hoverinfo='text',
            hovertext=text # This is the detailed text that will appear on hover
        ))

    # --- Configure Final Layout ---
    # Get the full company name for the title
    company_name_row = screener_data_df[screener_data_df['code'] == source_ticker]
    company_name = company_name_row['name'].iloc[0] if not company_name_row.empty else source_ticker # Use ticker if name not found
    fig.update_layout(
        title=f'Financial Solar System for {company_name} ({source_ticker})',
        title_x=0.5, # Center the title
        scene=dict(
            # Hide the 3D axes, grid, and background for a cleaner "space" look
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
            # Set the camera's position (eye) and the point it looks at (center)
            camera=dict(
                eye=dict(x=0.97, y=0.97, z=0.5),      # Camera position is below the plane
                center=dict(x=0, y=0, z=-0.3)       # Look at a point slightly above the origin
            ),
            # Use 'data' aspect mode to ensure spheres look like spheres and not ellipsoids
            aspectmode='data',
            # Add the text annotations to the scene
            annotations=scene_annotations
        ),
        margin=dict(l=0, r=0, b=0, t=40), # Reduce margins to maximize plot area
        showlegend=False, # We don't need a legend for this plot
        paper_bgcolor='black', # Background of the entire figure area
        plot_bgcolor='black', # Background of the plotting area itself
        # Apply the font to the entire plot (title, etc.) and increase size
        font=dict(
            family="'VT323', monospace",
            color='#39FF14',
            size=18
        )
    )
    return fig

# --- Dash App Definition ---
# Initialize the Dash application
app = Dash(__name__)

# --- Styling Dictionaries for the UI ---
# Reusable styles for a consistent "terminal" look and feel using the VT323 font
terminal_font = {'fontFamily': "'VT323', monospace", 'color': '#39FF14', 'fontSize': '18px'}
terminal_container_style = {'backgroundColor': 'black', 'border': '1px solid #39FF14', 'padding': '10px 15px', 'margin': '10px', 'borderRadius': '5px'}
terminal_header_style = {'color': '#00FFFF', 'textAlign': 'center', 'text-decoration': 'underline', 'marginTop': 0, 'marginBottom': '10px','fontFamily': "'VT323', monospace", 'fontSize': '22px'}
button_style_green = {**terminal_font, 'backgroundColor': '#33CC33', 'color': 'black', 'fontWeight': 'bold', 'border': 'none', 'padding': '5px 10px', 'margin': '3px 0', 'borderRadius': '3px', 'textAlign': 'center', 'width': '100%', 'fontSize': '16px'}
button_style_red = {**terminal_font, 'backgroundColor': '#FF0000', 'color': 'black', 'fontWeight': 'bold', 'border': 'none', 'padding': '5px 10px', 'margin': '3px 0', 'borderRadius': '3px', 'textAlign': 'center', 'width': '100%', 'fontSize': '16px'}


# Assume screener_data_df is loaded and available in the global scope
# Create the list of options for the dropdown menu from the screener data
ticker_options = [{'label': row['name'] + f" ({row['code']})", 'value': row['code']} for index, row in screener_data_df.iterrows()]
# Set a default ticker for the app to load initially
default_ticker = 'AAPL' if 'AAPL' in screener_data_df['code'].unique() else screener_data_df['code'].iloc[0]

# --- App Layout ---
# This defines the HTML structure of the web page.
app.layout = html.Div(style={'backgroundColor': 'black', 'color': '#39FF14', **terminal_font}, children=[
    # Import the Google Font
    html.Link(
        rel='stylesheet',
        href='https://fonts.googleapis.com/css2?family=VT323&display=swap'
    ),

    # Main title of the application
    html.H1("The Financial Planetarium", style={'textAlign': 'center', 'paddingTop': '20px', **terminal_font, 'fontSize': '36px', 'color': '#00FFFF'}),

    # Container for the dropdown menu
    html.Div([
        html.Label("Select a Stock:", style={'marginRight': '10px', **terminal_font}),
        dcc.Dropdown(
            id='ticker-dropdown', # Unique ID for this component
            options=ticker_options, # The list of tickers to choose from
            value=default_ticker, # The initial value
            style={'width': '300px', 'display': 'inline-block', **terminal_font, 'color': 'black'}
        ),
    ], style={'textAlign': 'center', 'padding': '20px'}),

    # Main content area with the graph and info tables, laid out horizontally
    html.Div([
        # --- Left Panel: Top Predictions ---
        html.Div([
            html.H3("Top Predictions", style=terminal_header_style),
            # Container for the two columns of buttons
            html.Div([
                # Column 1: Positive Buttons
                html.Div(
                    [html.Button(
                        f"{row['ticker']}: {row['gravitational_impact']:.2f}%",
                        id={'type': 'prediction-button', 'ticker': row['ticker']},
                        style=button_style_green
                    ) for index, row in top_positive_impacts.head(6).iterrows()],
                    style={'flex': 1, 'marginRight': '5px'}
                ),
                # Column 2: Negative Buttons
                html.Div(
                    [html.Button(
                        f"{row['ticker']}: {row['gravitational_impact']:.1f}%",
                        id={'type': 'prediction-button', 'ticker': row['ticker']},
                        style=button_style_red
                    ) for index, row in top_negative_impacts.head(6).iterrows()],
                    style={'flex': 1, 'marginLeft': '5px'}
                ),
            ], style={'display': 'flex', 'flexDirection': 'row'})
        ], style={**terminal_container_style, 'width': '20%', 'alignSelf': 'flex-start'}),

        # --- Center Panel: The Graph ---
        dcc.Graph(id='network-graph', style={'height': '50vh', 'width': '55%'}),

        # --- Right Panel: Info and Legend ---
        html.Div([
            html.Div(id='star-info-table', style=terminal_container_style),
            html.Div([
                html.H3("Legend", style=terminal_header_style),
                html.Div(style={
                    'height': '20px',
                    'width': '100%',
                    'background': f"linear-gradient(to right, {', '.join(colors)})",
                    'borderRadius': '3px'
                }),
                html.Div([
                    html.Span("Decrease", style={'textAlign': 'left', 'flex': 1}),
                    html.Span("Stable", style={'textAlign': 'center', 'flex': 1}),
                    html.Span("Increase", style={'textAlign': 'right', 'flex': 1})
                ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginTop': '5px'})
            ], style=terminal_container_style)
        ], style={'width': '25%', 'display': 'flex', 'flexDirection': 'column'})

    ], style={'display': 'flex', 'flexDirection': 'row'}),

    # A Div below the main content to hold the "Planet Information" table
    html.Div(id='planet-info-table', style={**terminal_container_style, 'marginTop': '20px'}),
])

# --- Callback for Prediction Buttons ---
@app.callback(
    Output('ticker-dropdown', 'value'),
    Input({'type': 'prediction-button', 'ticker': ALL}, 'n_clicks'),
    prevent_initial_call=True
)
def update_dropdown_from_button(n_clicks):
    # Check if any button was actually clicked
    if not any(n_clicks):
        raise PreventUpdate

    # Get the ID of the button that was clicked
    ctx = callback_context
    button_id_str = ctx.triggered[0]['prop_id'].split('.')[0]
    button_id = json.loads(button_id_str)

    # Return the ticker from the button's ID to update the dropdown
    return button_id['ticker']

# --- Callback to Update All Components ---
@app.callback(
    [Output('network-graph', 'figure'),
     Output('star-info-table', 'children'),
     Output('planet-info-table', 'children')],
    [Input('ticker-dropdown', 'value')]
)
def update_all_components(selected_ticker):
    """
    This function is triggered whenever the user selects a new ticker from the dropdown.
    It re-calculates and re-generates all the visual components of the app.
    """
    # NOTE: The data processing part is assumed to happen here.
    # In a real application, you would load or calculate your DataFrames.
    # The following line is an example and is commented out because it depends
    # on variables not defined in this standalone script.

    global processed_data_df, source_data_df, screener_data_df, six_month_spearman_lagged_correlations, three_month_spearman_lagged_correlations
    processed_data_df, source_data_df = process_and_score_stocks(
        six_month_spearman_lagged_correlations,
        three_month_spearman_lagged_correlations,
        screener_data_df,
        selected_ticker,
        min_nodes,
        max_nodes,
        threshold_percent
    )


    # Check if selected_ticker exists in screener_data_df and source_data_df
    if selected_ticker not in screener_data_df['code'].unique() or source_data_df.empty:
        empty_fig = go.Figure().update_layout(title=f"Data not available for {selected_ticker}", title_x=0.5, paper_bgcolor='black', font_color='#39FF14', font_family="'VT323', monospace")
        empty_star_table = [html.H3("Star Information", style=terminal_header_style), html.P(f"Data not available for {selected_ticker}", style=terminal_font)]
        empty_planet_table = [html.H3("Planet Information", style=terminal_header_style), html.P(f"Data not available for {selected_ticker}", style=terminal_font)]
        return empty_fig, empty_star_table, empty_planet_table


    # Generate the 3D plot using the main visualization function
    fig = solar_system_visual(selected_ticker, processed_data_df, source_data_df, screener_data_df)

    # --- Generate Star Information Table ---
    # Fetch the data for the selected star
    star_info_screener = screener_data_df[screener_data_df['code'] == selected_ticker].iloc[0]
    star_info_source = source_data_df[source_data_df['ticker'] == selected_ticker].iloc[0]
    market_cap = star_info_screener.get('market_capitalization', 0)
    market_cap_str = f"${market_cap/1e12:.2f}T" if market_cap > 1e12 else f"${market_cap/1e9:.2f}B"

    # Build the HTML components for the star info box
    star_table_children = [
        html.H3("Star Information", style=terminal_header_style),
        html.P(f"Name: {star_info_screener['name']} ({selected_ticker})", style=terminal_font),
        html.P(f"Market Cap: {market_cap_str}", style=terminal_font),
        html.P(f"Gravitational Impact: {star_info_source['gravitational_impact']:.2f}%", style=terminal_font),
    ]

    # --- Generate Planet Information Table ---
    # Get the data for all planets orbiting the selected star
    planets_df = processed_data_df[processed_data_df['source'] == selected_ticker]

    # --- Style definitions for the planet table ---
    # Style for the table header (solid green bar, black bold text, left-aligned)
    table_header_style = {
        **terminal_font, # Inherit base font styles
        'backgroundColor': '#39FF14',
        'color': 'black',
        'fontWeight': 'bold',
        'textAlign': 'left',
        'padding': '8px'
    }
    # Style for the table data cells (left-aligned text with padding)
    table_cell_style = {
        **terminal_font, # Inherit base font styles
        'textAlign': 'left',
        'padding': '8px',
        'borderBottom': '1px solid rgba(57, 255, 20, 0.3)' # Faint green line for row separation
    }

    # Define the table header using the new style
    table_header = [html.Thead(html.Tr([html.Th(col, style=table_header_style) for col in ["Ticker", "Name", "Market Cap", "Daily Change", "Correlation", "Gravitational Force"]]))]

    table_rows = []
    # Loop through the planet data to create a row for each one, applying the cell style
    for i, planet_row in planets_df.iterrows():
        screener_info_row = screener_data_df[screener_data_df['code'] == planet_row['target']]
        if screener_info_row.empty: continue # Skip if no screener data for planet

        screener_info = screener_info_row.iloc[0]
        planet_mcap_str = f"${screener_info['market_capitalization']/1e12:.2f}T" if screener_info['market_capitalization'] > 1e12 else f"${screener_info['market_capitalization']/1e9:.2f}B"
        table_rows.append(html.Tr([
            html.Td(planet_row['target'], style=table_cell_style),
            html.Td(screener_info['name'], style=table_cell_style),
            html.Td(planet_mcap_str, style=table_cell_style),
            html.Td(f"{planet_row['Daily Change']:.2f}%", style=table_cell_style),
            html.Td(f"{planet_row['unified_correlation']:.3f}", style=table_cell_style),
            html.Td(f"{planet_row['signed_gravitational_force']:.2f}", style=table_cell_style),
        ]))

    # Combine the header and rows into a full table
    planet_table_children = [
        html.H3("Planet Information", style=terminal_header_style),
        html.Table(table_header + [html.Tbody(table_rows)], style={'width': '100%', 'borderCollapse': 'collapse', 'borderSpacing': 0})
    ]

    # Return the generated figure and table contents to the appropriate 'Output' components
    return fig, star_table_children, planet_table_children

# --- Run the App ---
# This line is standard for running a Python script directly.
# The `if __name__ == '__main__':` block ensures this code only runs
# when the script is executed, not when it's imported as a module.
if __name__ == '__main__':
    # Starts the web server. `debug=True` allows for hot-reloading and provides error messages in the browser.
    app.run(debug=True, port=8051)


<IPython.core.display.Javascript object>

## Testing Ground 2

In [113]:
# This is a dash app by Chris Parks
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.colors as mcolors
from dash import Dash, dcc, html, callback_context
from dash.dependencies import Input, Output, State, ALL
from dash.exceptions import PreventUpdate
import json
import base64

# --- Helper Functions ---

def create_model_image_svg(base_color, subdivisions, texture_spots_count):
    """
    Creates a base64 encoded SVG data URL that is a 2D representation of the 3D low-poly model.
    This function generates the geometry, projects it, and simulates flat shading to match the plot.
    """
    # 1. Generate the 3D model's vertices and faces for a unit sphere
    # We pass 0 for texture_spots here because we'll draw them separately in the SVG
    vertices, faces, _ = create_low_poly_sphere(0, 0, 0, 1, base_color, subdivisions, 0)

    # 2. Define a light source for shading the facets
    light_source = np.array([-0.5, 0.8, 1.0])
    light_source = light_source / np.linalg.norm(light_source)

    # 3. Process each face for rendering
    face_data = []
    for face in faces:
        # Get the vertices for the current face
        v0, v1, v2 = vertices[face]

        # --- Back-face culling: Don't render faces pointing away from the camera ---
        # The camera is at (0, 0, z), so we check the z-component of the normal
        normal = np.cross(v1 - v0, v2 - v0)
        if np.linalg.norm(normal) == 0: continue
        normal = normal / np.linalg.norm(normal)
        if normal[2] < 0:
            continue # This face is on the back of the sphere, so we skip it

        # --- Shading: Calculate brightness based on angle to the light source ---
        intensity = np.dot(normal, light_source)
        # Map intensity to a brightness factor for the color
        color_factor = 0.65 + intensity * 0.5
        facet_color = darken_color(base_color, color_factor)

        # --- Projection: Convert 3D vertex coordinates to 2D SVG coordinates ---
        # We scale and shift the (x, y) coordinates to fit in a 100x100 SVG
        points_2d_str = " ".join([f"{(v[0] * 48) + 50},{(v[1] * -48) + 50}" for v in [v0, v1, v2]])

        # Store the face's z-depth for sorting, so closer faces draw on top
        avg_z = (v0[2] + v1[2] + v2[2]) / 3
        face_data.append({'z': avg_z, 'points': points_2d_str, 'color': facet_color})

    # 4. Sort faces from back to front
    face_data.sort(key=lambda f: f['z'])

    # 5. Build the SVG polygons from the sorted face data
    svg_polygons = "".join(f'<polygon points="{f["points"]}" fill="{f["color"]}" />' for f in face_data)

    # 6. Add texture spots as random circles
    texture_color = darken_color(base_color, 0.7)
    svg_texture_spots = ""
    np.random.seed(sum(ord(c) for c in base_color)) # Seed for consistency
    for _ in range(texture_spots_count):
        angle = np.random.uniform(0, 2 * np.pi)
        radius = np.random.uniform(0, 48)
        spot_size = np.random.uniform(4, 9)
        cx = 50 + radius * np.cos(angle)
        cy = 50 + radius * np.sin(angle)
        svg_texture_spots += f'<circle cx="{cx}" cy="{cy}" r="{spot_size}" fill="{texture_color}" opacity="0.7"/>'

    # 7. Assemble the final SVG string
    svg_string = f'''
    <svg width="100" height="100" viewBox="0 0 100 100" xmlns="http://www.w3.org/2000/svg">
      <defs>
        <clipPath id="sphereClip">
          <circle cx="50" cy="50" r="48"/>
        </clipPath>
        <filter id="blur-effect">
          <feGaussianBlur in="SourceGraphic" stdDeviation="0.7" />
        </filter>
      </defs>
      <g clip-path="url(#sphereClip)" filter="url(#blur-effect)">
        {svg_polygons}
        {svg_texture_spots}
      </g>
      <circle cx="50" cy="50" r="48" fill="none" stroke="rgba(255, 255, 255, 0.25)" stroke-width="1.5" />
    </svg>
    '''
    encoded_svg = base64.b64encode(svg_string.encode('utf-8')).decode('utf-8')
    return f"data:image/svg+xml;base64,{encoded_svg}"

def darken_color(color_hex, factor=0.8):
    """Darkens or lightens a hex color by a given factor."""
    rgb = mcolors.to_rgb(color_hex)
    # Clamp values to ensure they stay within the valid [0, 1] range for RGB
    modified_rgb = [min(max(c * factor, 0), 1) for c in rgb]
    return mcolors.to_hex(modified_rgb)

def create_low_poly_sphere(center_x, center_y, center_z, radius, base_color, subdivisions=2, texture_spots=15):
    """
    Generates vertex and face data for a textured low-poly sphere (icosphere).
    """
    # Define the 12 vertices of a regular icosahedron
    t = (1.0 + np.sqrt(5.0)) / 2.0
    vertices = np.array([
        [-1,  t,  0], [ 1,  t,  0], [-1, -t,  0], [ 1, -t,  0],
        [ 0, -1,  t], [ 0,  1,  t], [ 0, -1, -t], [ 0,  1, -t],
        [ t,  0, -1], [ t,  0,  1], [-t,  0, -1], [-t,  0,  1]
    ])

    # Define the 20 triangular faces of the icosahedron
    faces = np.array([
        [0, 11, 5], [0, 5, 1], [0, 1, 7], [0, 7, 10], [0, 10, 11],
        [1, 5, 9], [5, 11, 4], [11, 10, 2], [10, 7, 6], [7, 1, 8],
        [3, 9, 4], [3, 4, 2], [3, 2, 6], [3, 6, 8], [3, 8, 9],
        [4, 9, 5], [2, 4, 11], [6, 2, 10], [8, 6, 7], [9, 8, 1]
    ])

    # Subdivide faces to create more polygons
    for _ in range(subdivisions):
        new_faces = []
        mid_points = {}
        for face in faces:
            v_indices = [face[0], face[1], face[2]]
            new_v_indices = []
            for i in range(3):
                v1 = v_indices[i]
                v2 = v_indices[(i + 1) % 3]
                mid_key = tuple(sorted((v1, v2)))
                mid_idx = mid_points.get(mid_key)
                if mid_idx is None:
                    mid_idx = len(vertices)
                    vertices = np.vstack([vertices, (vertices[v1] + vertices[v2]) / 2.0])
                    mid_points[mid_key] = mid_idx
                new_v_indices.append(mid_idx)
            new_faces.append([v_indices[0], new_v_indices[0], new_v_indices[2]])
            new_faces.append([v_indices[1], new_v_indices[1], new_v_indices[0]])
            new_faces.append([v_indices[2], new_v_indices[2], new_v_indices[1]])
            new_faces.append(new_v_indices)
        faces = np.array(new_faces)

    # Normalize vertices to form a sphere, then scale and translate
    vertices = vertices / np.linalg.norm(vertices, axis=1)[:, np.newaxis]
    final_vertices = vertices * radius + np.array([center_x, center_y, center_z])

    # Create vertex colors for texture spots
    darker_color_hex = darken_color(base_color, 0.7)
    vertex_colors = [base_color] * len(final_vertices)
    if texture_spots > 0:
        spot_indices = np.random.choice(len(final_vertices), size=texture_spots, replace=False)
        for idx in spot_indices:
            vertex_colors[idx] = darker_color_hex

    return final_vertices, faces, vertex_colors

# --- Define New Color Palette and Colormap ---
# Define the red and green color spectrums
RED_SPECTRUM = {'light': '#ee868f', 'dark': '#cd2d35'}
GREEN_SPECTRUM = {'light': '#8ac08b', 'dark': '#329542'}

# Create separate colormaps for negative (red) and positive (green) values
red_cmap = mcolors.LinearSegmentedColormap.from_list('red_cmap', [RED_SPECTRUM['light'], RED_SPECTRUM['dark']])
green_cmap = mcolors.LinearSegmentedColormap.from_list('green_cmap', [GREEN_SPECTRUM['light'], GREEN_SPECTRUM['dark']])

def get_node_color(value, min_val, max_val):
    """
    Determines the node color based on its value, using a diverging red/green scale.
    """
    if value >= 0:
        if value >= max_val:
            return GREEN_SPECTRUM['dark']
        norm_val = value / max_val if max_val != 0 else 0
        return mcolors.to_hex(green_cmap(norm_val))
    else: # value < 0
        if value <= min_val:
            return RED_SPECTRUM['dark']
        norm_val = value / min_val if min_val != 0 else 0
        return mcolors.to_hex(red_cmap(norm_val))


# --- Main Plotting Function ---
def solar_system_visual(source_ticker, processed_data_df, source_data_df, screener_data_df):
    """
    Creates the main 3D solar system visualization with a low-poly aesthetic.
    """
    ticker_connections = processed_data_df[processed_data_df['source'] == source_ticker].copy()
    source_info_row = source_data_df[source_data_df['ticker'] == source_ticker]

    if ticker_connections.empty or source_info_row.empty:
        return go.Figure().update_layout(title=f"Data not available for {source_ticker}", title_x=0.5, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font_color='white')

    source_info = source_info_row.iloc[0]
    fig = go.Figure()

    pos = {source_ticker: (0, 0, 0)}
    actual_target_connections = ticker_connections[ticker_connections['target'] != source_ticker].copy()
    num_connections = len(actual_target_connections)
    radii_for_rings = []

    if num_connections > 0:
        original_radii = actual_target_connections['Orbital Radius']
        min_rad, max_rad = original_radii.min(), original_radii.max()
        rad_range = max_rad - min_rad if max_rad > min_rad else 1.0
        min_visual_radius, max_visual_radius = 3.0, 10.0
        visual_range = max_visual_radius - min_visual_radius
        thetas = np.linspace(0, 2 * np.pi, num_connections, endpoint=False)

        for i, (index, row) in enumerate(actual_target_connections.iterrows()):
            scaled_radius = ((row['Orbital Radius'] - min_rad) / rad_range) * visual_range + min_visual_radius
            radii_for_rings.append(scaled_radius)
            theta = thetas[i]
            pos[row['target']] = (scaled_radius * np.cos(theta), scaled_radius * np.sin(theta), 0)

    # --- Add Orbital Rings ---
    for r in sorted(list(set(radii_for_rings))):
        theta_ring = np.linspace(0, 2 * np.pi, 100)
        fig.add_trace(go.Scatter3d(
            x=r * np.cos(theta_ring), y=r * np.sin(theta_ring), z=np.zeros(100),
            mode='lines',
            line=dict(color='rgba(255, 255, 255, 0.2)', width=1, dash='solid'),
            hoverinfo='none'
        ))

    scene_annotations = []

    # --- Loop Through Each Node to Draw Them ---
    for node_name, coords in pos.items():
        center_x, center_y, center_z = coords
        is_source = (node_name == source_ticker)

        screener_info_row = screener_data_df[screener_data_df['code'] == node_name]
        if screener_info_row.empty: continue
        screener_info = screener_info_row.iloc[0]

        market_cap = screener_info.get('market_capitalization', 0)
        market_cap_str = f"${market_cap/1e12:.2f}T" if market_cap > 1e12 else f"${market_cap/1e9:.2f}B"
        min_visual_size, max_visual_size = 0.6, 1.5

        if is_source:
            hover_text = (f"<b>{screener_info.get('name', node_name)} ({node_name})</b><br>"
                          f"Industry: {screener_info.get('industry', 'N/A')}<br>"
                          f"Sector: {screener_info.get('sector', 'N/A')}<br>"
                          f"Avg Volume (1d): {screener_info.get('avgvol_1d', 'N/A')}<br>"
                          f"Market Cap: {market_cap_str}")
            node_color = get_node_color(source_info['gravitational_impact'], -80, 80)
            radius = min_visual_size + (source_info['source_planet_radius'] * (max_visual_size - min_visual_size))
            subdivisions = 2
        else:
            processed_info = ticker_connections[ticker_connections['target'] == node_name].iloc[0]
            hover_text = (f"<b>{screener_info.get('name', node_name)} ({node_name})</b><br>"
                          f"Industry: {screener_info.get('industry', 'N/A')}<br>"
                          f"Sector: {screener_info.get('sector', 'N/A')}<br>"
                          f"Avg Volume (1d): {screener_info.get('avgvol_1d', 'N/A')}<br>"
                          f"Daily Change: {processed_info['Daily Change']:.2f}%<br>"
                          f"Market Cap: {market_cap_str}")
            node_color = get_node_color(processed_info['Daily Change'], -5, 5)
            radius = min_visual_size + (processed_info['Planet Radius'] * (max_visual_size - min_visual_size))
            subdivisions = 2

        vertices, faces, vertex_colors = create_low_poly_sphere(center_x, center_y, center_z, radius, node_color, subdivisions, texture_spots=15)
        fig.add_trace(go.Mesh3d(
            x=vertices[:, 0], y=vertices[:, 1], z=vertices[:, 2],
            i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
            vertexcolor=vertex_colors,
            opacity=1.0,
            flatshading=True,
            # Add lighting properties to make edges crisp
            lighting=dict(
                ambient=0.8,
                diffuse=0.5,
                specular=1,
                roughness=1
            ),
            hoverinfo='text',
            text=hover_text,
            hoverlabel=dict(bgcolor='#0f0524', font=dict(color='#EAEAEA', size=14), bordercolor='rgba(255, 255, 255, 0.3)')
        ))

        # Add labels as scene annotations
        scene_annotations.append(
            dict(
                x=center_x,
                y=center_y,
                z=center_z, # Position label in the center of the sphere
                text=f"<b>{node_name}</b>",
                showarrow=False,
                font=dict(color='white', size=14),
                bgcolor="rgba(0,0,0,0)",
                xanchor="center"
            )
        )


    # --- Configure Final Layout ---
    fig.update_layout(
        scene=dict(
            xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False),
            camera=dict(eye=dict(x=0.9, y=0.9, z=0.9)),
            aspectmode='data',
            annotations=scene_annotations,
            bgcolor='rgba(0,0,0,0)'
        ),
        margin=dict(l=0, r=0, b=0, t=0),
        showlegend=False,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        font=dict(family="'Space Grotesk', sans-serif", color='#EAEAEA', size=16)
    )
    return fig

# --- Dash App Definition & Styling ---
app = Dash(__name__, external_stylesheets=['https://fonts.googleapis.com/css2?family=Space+Grotesk:wght@400;700&display=swap'])
server = app.server

# --- New Theme Styles ---
THEME = {
    'background': '#0B041A',
    'text': '#EAEAEA',
    'primary': '#FFFFFF',
    'container_bg': 'rgba(30, 15, 60, 0.3)',
    'container_border': 'rgba(255, 255, 255, 0.1)'
}

# Starry background style
starry_background_style = {
    'backgroundColor': THEME['background'],
    'backgroundImage': 'radial-gradient(circle, white 0.5px, transparent 1.5px), radial-gradient(circle, white 1px, transparent 2px), radial-gradient(circle, white 0.5px, transparent 1.5px)',
    'backgroundSize': '350px 350px, 250px 250px, 150px 150px',
    'backgroundPosition': '0 0, 40px 60px, 130px 270px',
    'color': THEME['text'],
    'fontFamily': "'Space Grotesk', sans-serif",
    'minHeight': '100vh',
    'padding': '20px'
}

font_style = {'fontFamily': "'Space Grotesk', sans-serif", 'color': THEME['text']}
container_style = {
    'backgroundColor': THEME['container_bg'],
    'border': f"1px solid {THEME['container_border']}",
    'padding': '25px 30px',
    'borderRadius': '12px',
    'backdropFilter': 'blur(10px)',
    'width': '100%',
    'boxSizing': 'border-box'
}
header_style = {**font_style, 'color': THEME['text'], 'textAlign': 'center', 'fontWeight': 'bold', 'marginTop': 0, 'marginBottom': '20px', 'fontSize': '22px'}

# Assume dataframes are loaded
ticker_options = [{'label': row['name'] + f" ({row['code']})", 'value': row['code']} for index, row in screener_data_df.iterrows()]
default_ticker = 'AAPL'

# --- App Layout ---
app.layout = html.Div(style=starry_background_style, children=[
    html.H1("THE FINANCIAL PLANETARIUM", style={**font_style, 'textAlign': 'center', 'padding': '20px 0', 'fontSize': '32px', 'fontWeight': 'bold', 'letterSpacing': '4px'}),

    html.Div([
        dcc.Dropdown(id='ticker-dropdown', options=ticker_options, value=default_ticker, clearable=False, style={'height': '40px', 'color': 'black'})
    ], style={'width': '90%', 'maxWidth': '500px', 'margin': '0 auto 20px auto', 'backgroundColor': THEME['container_bg'], 'border': f"1px solid {THEME['container_border']}", 'borderRadius': '12px', 'backdropFilter': 'blur(10px)', 'padding': '5px'}),

    html.P(id='prediction-summary-text', style={'textAlign': 'center', 'padding': '10px 0', 'fontSize': '18px', 'color': THEME['text']}),
    html.Div(id='graph-container', style={'height': '65vh', 'width': '98%', 'margin': 'auto', 'borderRadius': '15px', 'boxShadow': '0 0 25px 5px rgba(255, 255, 255, 0.15)'}),
    html.Div(id='info-panels-container')
])


# --- Main Callback to Generate Content ---
@app.callback(
    [Output('graph-container', 'children'),
     Output('info-panels-container', 'children'),
     Output('prediction-summary-text', 'children')],
    Input('ticker-dropdown', 'value')
)
def update_layout_and_data(selected_ticker):

    global processed_data_df, source_data_df
    processed_data_df, source_data_df = process_and_score_stocks(
        six_month_spearman_lagged_correlations, three_month_spearman_lagged_correlations, screener_data_df,
        selected_ticker, min_nodes, max_nodes, threshold_percent
    )

    if processed_data_df.empty or source_data_df.empty:
        empty_fig = go.Figure().update_layout(title=f"Data not available for {selected_ticker}", title_x=0.5, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font_color='white')
        return dcc.Graph(id='network-graph', figure=empty_fig, style={'height': '100%'}), html.Div(f"Data not available for {selected_ticker}", style={'textAlign': 'center', 'padding': '20px'}), ""

    graph = dcc.Graph(id='network-graph', figure=solar_system_visual(selected_ticker, processed_data_df, source_data_df, screener_data_df), style={'height': '100%'})

    # --- Generate data for panels and summary text ---
    star_info_screener = screener_data_df[screener_data_df['code'] == selected_ticker].iloc[0]
    star_info_source = source_data_df[source_data_df['ticker'] == selected_ticker].iloc[0]
    grav_impact = star_info_source.get('gravitational_impact', 0)

    # Summary text
    source_name = star_info_screener.get('name', selected_ticker)
    direction = "increase" if grav_impact >= 0 else "decrease"
    strength = grav_impact
    prediction_summary = f"{source_name} ({selected_ticker}) is predicted to {direction} today with a prediction strength of {strength:.2f}%."

    # Star Info Panel
    star_color = get_node_color(grav_impact, -80, 80)
    star_image_src = create_model_image_svg(star_color, subdivisions=2, texture_spots_count=10)
    star_text_content = html.Div([
        html.P(f"Name: {star_info_screener['name']} ({selected_ticker})", style={'margin': '0 0 5px 0'}),
        html.P(f"Market Cap: ${star_info_screener.get('market_capitalization', 0)/1e12:.2f}T", style={'margin': '0 0 5px 0'}),
        html.P(f"Gravitational Impact: {grav_impact:.2f}%", style={'margin': '0'}),
    ])
    star_info_panel = html.Div([
        html.H3("Star Information", style=header_style),
        html.Div([
            html.Img(src=star_image_src, style={'height': '150px', 'width': '150px', 'marginRight': '20px'}),
            star_text_content
        ], style={'display': 'flex', 'alignItems': 'center'})
    ], style=container_style)

    # --- Predictions Panel ---
    prediction_items = []
    if 'top_positive_impacts' in globals() and not top_positive_impacts.empty and 'top_negative_impacts' in globals() and not top_negative_impacts.empty:
        combined_impacts = pd.concat([top_positive_impacts.head(5), top_negative_impacts.head(5)])
        prediction_items = [
            html.Div([
                html.Span(row['ticker']),
                html.Span(f"{row['gravitational_impact']:.2f}%", style={'color': '#4ade80' if row['gravitational_impact'] > 0 else '#f87171', 'fontWeight': 'bold'})
            ], style={'display': 'flex', 'justifyContent': 'space-between', 'padding': '8px 0', 'borderBottom': f'1px solid {THEME["container_border"]}'})
            for _, row in combined_impacts.iterrows()
        ]
    else:
         prediction_items = [html.Div("Top predictions not available.")]
    predictions_panel = html.Div([html.H3("Top Predictions", style=header_style), html.Div(prediction_items)], style=container_style)

    # --- Legend Panel ---
    legend_gradient_style = {'height': '15px', 'width': '100%', 'maxWidth': '300px', 'borderRadius': '5px', 'marginBottom': '8px', 'background': f"linear-gradient(to right, {RED_SPECTRUM['dark']}, {RED_SPECTRUM['light']} 49.9%, {GREEN_SPECTRUM['light']} 50.1%, {GREEN_SPECTRUM['dark']})"}
    legend_content = html.Div([
        html.Div(style=legend_gradient_style),
        html.Div([html.Span("Decrease"), html.Span("Increase")], style={'display': 'flex', 'justifyContent': 'space-between', 'fontSize': '12px', 'width': '100%', 'maxWidth': '300px'})
    ], style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center'})
    legend_panel_container = html.Div([
        html.H3("Legend", style={**header_style, 'marginBottom': '15px', 'textAlign': 'center'}),
        legend_content
    ], style={**container_style, 'padding': '15px 20px', 'width': '100%', 'align-self': 'center'})

    # --- Planet Info Table ---
    planets_df = processed_data_df[processed_data_df['source'] == selected_ticker].copy()
    table_header = [html.Thead(html.Tr([html.Th(col, style={'padding': '12px', 'textAlign': 'left', 'borderBottom': f"2px solid {THEME['container_border']}"}) for col in ["Ticker", "Name", "Market Cap", "Daily Change", "Correlation", "Grav. Force"]]))]
    table_rows = []
    if not planets_df.empty:
        for _, p_row in planets_df.iterrows():
            s_info = screener_data_df[screener_data_df['code'] == p_row['target']].iloc[0]
            planet_color = get_node_color(p_row['Daily Change'], -5, 5)
            planet_image_src = create_model_image_svg(planet_color, subdivisions=2, texture_spots_count=5)

            ticker_cell = html.Div([
                html.Img(src=planet_image_src, style={'height': '40px', 'width': '40px', 'marginRight': '10px'}),
                html.Span(p_row['target'])
            ], style={'display': 'flex', 'alignItems': 'center'})

            table_rows.append(html.Tr([
                html.Td(ticker_cell, style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(s_info['name'], style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(f"${s_info['market_capitalization']/1e9:.2f}B", style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(f"{p_row['Daily Change']:.2f}%", style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(f"{p_row['unified_correlation']:.3f}", style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(f"{p_row['signed_gravitational_force']:.2f}", style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
            ]))

    # Dynamically set the height of the table container
    table_wrapper_style = {'overflowX': 'auto'}
    if len(planets_df) > 5:
        table_wrapper_style['maxHeight'] = '300px'
        table_wrapper_style['overflowY'] = 'auto'

    planet_table_panel = html.Div([
        html.H3("Planet Information", style=header_style),
        html.Div(html.Table(table_header + [html.Tbody(table_rows)], style={'width': '100%', 'borderCollapse': 'collapse'}), style=table_wrapper_style)
    ], style=container_style)

    # Assemble the info panels layout
    info_panels = html.Div([
        legend_panel_container,
        star_info_panel,
        planet_table_panel,
        predictions_panel
    ], style={'display': 'flex', 'flexDirection': 'column', 'gap': '20px', 'padding': '20px 0'})

    return graph, info_panels, prediction_summary

# --- Run the App ---
if __name__ == '__main__':
    app.run(debug=True, port=8051)


<IPython.core.display.Javascript object>

In [110]:
# This is a dash app by Chris Parks
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.colors as mcolors
from dash import Dash, dcc, html, callback_context
from dash.dependencies import Input, Output, State, ALL
from dash.exceptions import PreventUpdate
import json
import base64

# --- Helper Functions ---

def create_model_image_svg(base_color, subdivisions, texture_spots_count):
    """
    Creates a base64 encoded SVG data URL that is a 2D representation of the 3D low-poly model.
    This function generates the geometry, projects it, and simulates flat shading to match the plot.
    """
    # 1. Generate the 3D model's vertices and faces for a unit sphere
    # We pass 0 for texture_spots here because we'll draw them separately in the SVG
    vertices, faces, _ = create_low_poly_sphere(0, 0, 0, 1, base_color, subdivisions, 0)

    # 2. Define a light source for shading the facets
    light_source = np.array([-0.5, 0.8, 1.0])
    light_source = light_source / np.linalg.norm(light_source)

    # 3. Process each face for rendering
    face_data = []
    for face in faces:
        # Get the vertices for the current face
        v0, v1, v2 = vertices[face]

        # --- Back-face culling: Don't render faces pointing away from the camera ---
        # The camera is at (0, 0, z), so we check the z-component of the normal
        normal = np.cross(v1 - v0, v2 - v0)
        if np.linalg.norm(normal) == 0: continue
        normal = normal / np.linalg.norm(normal)
        if normal[2] < 0:
            continue # This face is on the back of the sphere, so we skip it

        # --- Shading: Calculate brightness based on angle to the light source ---
        intensity = np.dot(normal, light_source)
        # Map intensity to a brightness factor for the color
        color_factor = 0.65 + intensity * 0.5
        facet_color = darken_color(base_color, color_factor)

        # --- Projection: Convert 3D vertex coordinates to 2D SVG coordinates ---
        # We scale and shift the (x, y) coordinates to fit in a 100x100 SVG
        points_2d_str = " ".join([f"{(v[0] * 48) + 50},{(v[1] * -48) + 50}" for v in [v0, v1, v2]])

        # Store the face's z-depth for sorting, so closer faces draw on top
        avg_z = (v0[2] + v1[2] + v2[2]) / 3
        face_data.append({'z': avg_z, 'points': points_2d_str, 'color': facet_color})

    # 4. Sort faces from back to front
    face_data.sort(key=lambda f: f['z'])

    # 5. Build the SVG polygons from the sorted face data
    svg_polygons = "".join(f'<polygon points="{f["points"]}" fill="{f["color"]}" />' for f in face_data)

    # 6. Add texture spots as random circles
    texture_color = darken_color(base_color, 0.7)
    svg_texture_spots = ""
    np.random.seed(sum(ord(c) for c in base_color)) # Seed for consistency
    for _ in range(texture_spots_count):
        angle = np.random.uniform(0, 2 * np.pi)
        radius = np.random.uniform(0, 48)
        spot_size = np.random.uniform(4, 9)
        cx = 50 + radius * np.cos(angle)
        cy = 50 + radius * np.sin(angle)
        svg_texture_spots += f'<circle cx="{cx}" cy="{cy}" r="{spot_size}" fill="{texture_color}" opacity="0.7"/>'

    # 7. Assemble the final SVG string
    svg_string = f'''
    <svg width="100" height="100" viewBox="0 0 100 100" xmlns="http://www.w3.org/2000/svg">
      <defs>
        <clipPath id="sphereClip">
          <circle cx="50" cy="50" r="48"/>
        </clipPath>
        <filter id="blur-effect">
          <feGaussianBlur in="SourceGraphic" stdDeviation="0.7" />
        </filter>
      </defs>
      <g clip-path="url(#sphereClip)" filter="url(#blur-effect)">
        {svg_polygons}
        {svg_texture_spots}
      </g>
      <circle cx="50" cy="50" r="48" fill="none" stroke="rgba(255, 255, 255, 0.25)" stroke-width="1.5" />
    </svg>
    '''
    encoded_svg = base64.b64encode(svg_string.encode('utf-8')).decode('utf-8')
    return f"data:image/svg+xml;base64,{encoded_svg}"

def darken_color(color_hex, factor=0.8):
    """Darkens or lightens a hex color by a given factor."""
    rgb = mcolors.to_rgb(color_hex)
    # Clamp values to ensure they stay within the valid [0, 1] range for RGB
    modified_rgb = [min(max(c * factor, 0), 1) for c in rgb]
    return mcolors.to_hex(modified_rgb)

def create_low_poly_sphere(center_x, center_y, center_z, radius, base_color, subdivisions=2, texture_spots=15):
    """
    Generates vertex and face data for a textured low-poly sphere (icosphere).
    """
    # Define the 12 vertices of a regular icosahedron
    t = (1.0 + np.sqrt(5.0)) / 2.0
    vertices = np.array([
        [-1,  t,  0], [ 1,  t,  0], [-1, -t,  0], [ 1, -t,  0],
        [ 0, -1,  t], [ 0,  1,  t], [ 0, -1, -t], [ 0,  1, -t],
        [ t,  0, -1], [ t,  0,  1], [-t,  0, -1], [-t,  0,  1]
    ])

    # Define the 20 triangular faces of the icosahedron
    faces = np.array([
        [0, 11, 5], [0, 5, 1], [0, 1, 7], [0, 7, 10], [0, 10, 11],
        [1, 5, 9], [5, 11, 4], [11, 10, 2], [10, 7, 6], [7, 1, 8],
        [3, 9, 4], [3, 4, 2], [3, 2, 6], [3, 6, 8], [3, 8, 9],
        [4, 9, 5], [2, 4, 11], [6, 2, 10], [8, 6, 7], [9, 8, 1]
    ])

    # Subdivide faces to create more polygons
    for _ in range(subdivisions):
        new_faces = []
        mid_points = {}
        for face in faces:
            v_indices = [face[0], face[1], face[2]]
            new_v_indices = []
            for i in range(3):
                v1 = v_indices[i]
                v2 = v_indices[(i + 1) % 3]
                mid_key = tuple(sorted((v1, v2)))
                mid_idx = mid_points.get(mid_key)
                if mid_idx is None:
                    mid_idx = len(vertices)
                    vertices = np.vstack([vertices, (vertices[v1] + vertices[v2]) / 2.0])
                    mid_points[mid_key] = mid_idx
                new_v_indices.append(mid_idx)
            new_faces.append([v_indices[0], new_v_indices[0], new_v_indices[2]])
            new_faces.append([v_indices[1], new_v_indices[1], new_v_indices[0]])
            new_faces.append([v_indices[2], new_v_indices[2], new_v_indices[1]])
            new_faces.append(new_v_indices)
        faces = np.array(new_faces)

    # Normalize vertices to form a sphere, then scale and translate
    vertices = vertices / np.linalg.norm(vertices, axis=1)[:, np.newaxis]
    final_vertices = vertices * radius + np.array([center_x, center_y, center_z])

    # Create vertex colors for texture spots
    darker_color_hex = darken_color(base_color, 0.7)
    vertex_colors = [base_color] * len(final_vertices)
    if texture_spots > 0:
        spot_indices = np.random.choice(len(final_vertices), size=texture_spots, replace=False)
        for idx in spot_indices:
            vertex_colors[idx] = darker_color_hex

    return final_vertices, faces, vertex_colors

# --- Define New Color Palette and Colormap ---
# Define the red and green color spectrums
RED_SPECTRUM = {'light': '#ee868f', 'dark': '#cd2d35'}
GREEN_SPECTRUM = {'light': '#8ac08b', 'dark': '#329542'}

# Create separate colormaps for negative (red) and positive (green) values
red_cmap = mcolors.LinearSegmentedColormap.from_list('red_cmap', [RED_SPECTRUM['light'], RED_SPECTRUM['dark']])
green_cmap = mcolors.LinearSegmentedColormap.from_list('green_cmap', [GREEN_SPECTRUM['light'], GREEN_SPECTRUM['dark']])

def get_node_color(value, min_val, max_val):
    """
    Determines the node color based on its value, using a diverging red/green scale.
    """
    if value >= 0:
        if value >= max_val:
            return GREEN_SPECTRUM['dark']
        norm_val = value / max_val if max_val != 0 else 0
        return mcolors.to_hex(green_cmap(norm_val))
    else: # value < 0
        if value <= min_val:
            return RED_SPECTRUM['dark']
        norm_val = value / min_val if min_val != 0 else 0
        return mcolors.to_hex(red_cmap(norm_val))


# --- Main Plotting Function ---
def solar_system_visual(source_ticker, processed_data_df, source_data_df, screener_data_df):
    """
    Creates the main 3D solar system visualization with a low-poly aesthetic.
    """
    ticker_connections = processed_data_df[processed_data_df['source'] == source_ticker].copy()
    source_info_row = source_data_df[source_data_df['ticker'] == source_ticker]

    if ticker_connections.empty or source_info_row.empty:
        return go.Figure().update_layout(title=f"Data not available for {source_ticker}", title_x=0.5, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font_color='white')

    source_info = source_info_row.iloc[0]
    fig = go.Figure()

    pos = {source_ticker: (0, 0, 0)}
    actual_target_connections = ticker_connections[ticker_connections['target'] != source_ticker].copy()
    num_connections = len(actual_target_connections)
    radii_for_rings = []

    if num_connections > 0:
        original_radii = actual_target_connections['Orbital Radius']
        min_rad, max_rad = original_radii.min(), original_radii.max()
        rad_range = max_rad - min_rad if max_rad > min_rad else 1.0
        min_visual_radius, max_visual_radius = 3.0, 10.0
        visual_range = max_visual_radius - min_visual_radius
        thetas = np.linspace(0, 2 * np.pi, num_connections, endpoint=False)

        for i, (index, row) in enumerate(actual_target_connections.iterrows()):
            scaled_radius = ((row['Orbital Radius'] - min_rad) / rad_range) * visual_range + min_visual_radius
            radii_for_rings.append(scaled_radius)
            theta = thetas[i]
            pos[row['target']] = (scaled_radius * np.cos(theta), scaled_radius * np.sin(theta), 0)

    # --- Add Orbital Rings ---
    for r in sorted(list(set(radii_for_rings))):
        theta_ring = np.linspace(0, 2 * np.pi, 100)
        fig.add_trace(go.Scatter3d(
            x=r * np.cos(theta_ring), y=r * np.sin(theta_ring), z=np.zeros(100),
            mode='lines',
            line=dict(color='rgba(255, 255, 255, 0.2)', width=1, dash='solid'),
            hoverinfo='none'
        ))

    scene_annotations = []

    # --- Loop Through Each Node to Draw Them ---
    for node_name, coords in pos.items():
        center_x, center_y, center_z = coords
        is_source = (node_name == source_ticker)

        screener_info_row = screener_data_df[screener_data_df['code'] == node_name]
        if screener_info_row.empty: continue
        screener_info = screener_info_row.iloc[0]

        market_cap = screener_info.get('market_capitalization', 0)
        market_cap_str = f"${market_cap/1e12:.2f}T" if market_cap > 1e12 else f"${market_cap/1e9:.2f}B"
        min_visual_size, max_visual_size = 0.6, 1.5

        if is_source:
            hover_text = (f"<b>{screener_info.get('name', node_name)} ({node_name})</b><br>"
                          f"Industry: {screener_info.get('industry', 'N/A')}<br>"
                          f"Sector: {screener_info.get('sector', 'N/A')}<br>"
                          f"Avg Volume (1d): {screener_info.get('avgvol_1d', 'N/A')}<br>"
                          f"Market Cap: {market_cap_str}")
            node_color = get_node_color(source_info['gravitational_impact'], -80, 80)
            radius = min_visual_size + (source_info['source_planet_radius'] * (max_visual_size - min_visual_size))
            subdivisions = 2
        else:
            processed_info = ticker_connections[ticker_connections['target'] == node_name].iloc[0]
            hover_text = (f"<b>{screener_info.get('name', node_name)} ({node_name})</b><br>"
                          f"Industry: {screener_info.get('industry', 'N/A')}<br>"
                          f"Sector: {screener_info.get('sector', 'N/A')}<br>"
                          f"Avg Volume (1d): {screener_info.get('avgvol_1d', 'N/A')}<br>"
                          f"Daily Change: {processed_info['Daily Change']:.2f}%<br>"
                          f"Market Cap: {market_cap_str}")
            node_color = get_node_color(processed_info['Daily Change'], -5, 5)
            radius = min_visual_size + (processed_info['Planet Radius'] * (max_visual_size - min_visual_size))
            subdivisions = 2

        vertices, faces, vertex_colors = create_low_poly_sphere(center_x, center_y, center_z, radius, node_color, subdivisions, texture_spots=15)
        fig.add_trace(go.Mesh3d(
            x=vertices[:, 0], y=vertices[:, 1], z=vertices[:, 2],
            i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
            vertexcolor=vertex_colors,
            opacity=1.0,
            flatshading=True,
            # Add lighting properties to make edges crisp
            lighting=dict(
                ambient=0.8,
                diffuse=0.5,
                specular=1,
                roughness=1
            ),
            hoverinfo='text',
            text=hover_text,
            hoverlabel=dict(bgcolor='#0f0524', font=dict(color='#EAEAEA', size=14), bordercolor='rgba(255, 255, 255, 0.3)')
        ))

        # Add labels as scene annotations
        scene_annotations.append(
            dict(
                x=center_x,
                y=center_y,
                z=center_z, # Position label in the center of the sphere
                text=f"<b>{node_name}</b>",
                showarrow=False,
                font=dict(color='white', size=14),
                bgcolor="rgba(0,0,0,0)",
                xanchor="center"
            )
        )


    # --- Configure Final Layout ---
    fig.update_layout(
        scene=dict(
            xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False),
            camera=dict(eye=dict(x=0.9, y=0.9, z=0.9)),
            aspectmode='data',
            annotations=scene_annotations,
            bgcolor='rgba(0,0,0,0)'
        ),
        margin=dict(l=0, r=0, b=0, t=0),
        showlegend=False,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        font=dict(family="'Space Grotesk', sans-serif", color='#EAEAEA', size=16)
    )
    return fig

# --- Dash App Definition & Styling ---
app = Dash(__name__, external_stylesheets=['https://fonts.googleapis.com/css2?family=Space+Grotesk:wght@400;700&display=swap'])
server = app.server

# --- New Theme Styles ---
THEME = {
    'background': '#0B041A',
    'text': '#EAEAEA',
    'primary': '#FFFFFF',
    'container_bg': 'rgba(30, 15, 60, 0.3)',
    'container_border': 'rgba(255, 255, 255, 0.1)'
}

# Starry background style
starry_background_style = {
    'backgroundColor': THEME['background'],
    'backgroundImage': 'radial-gradient(circle, white 0.5px, transparent 1.5px), radial-gradient(circle, white 1px, transparent 2px), radial-gradient(circle, white 0.5px, transparent 1.5px)',
    'backgroundSize': '350px 350px, 250px 250px, 150px 150px',
    'backgroundPosition': '0 0, 40px 60px, 130px 270px',
    'color': THEME['text'],
    'fontFamily': "'Space Grotesk', sans-serif",
    'minHeight': '100vh',
    'padding': '20px'
}

font_style = {'fontFamily': "'Space Grotesk', sans-serif", 'color': THEME['text']}
container_style = {
    'backgroundColor': THEME['container_bg'],
    'border': f"1px solid {THEME['container_border']}",
    'padding': '25px 30px',
    'borderRadius': '12px',
    'backdropFilter': 'blur(10px)',
    'width': '100%',
    'boxSizing': 'border-box'
}
header_style = {**font_style, 'color': THEME['text'], 'textAlign': 'center', 'fontWeight': 'bold', 'marginTop': 0, 'marginBottom': '20px', 'fontSize': '22px'}

# Assume dataframes are loaded
ticker_options = [{'label': row['name'] + f" ({row['code']})", 'value': row['code']} for index, row in screener_data_df.iterrows()]
default_ticker = 'AAPL'

# --- App Layout ---
app.layout = html.Div(style=starry_background_style, children=[
    html.H1("THE FINANCIAL PLANETARIUM", style={**font_style, 'textAlign': 'center', 'padding': '20px 0', 'fontSize': '32px', 'fontWeight': 'bold', 'letterSpacing': '4px'}),

    html.Div([
        dcc.Dropdown(id='ticker-dropdown', options=ticker_options, value=default_ticker, clearable=False, style={'height': '40px', 'color': 'black'})
    ], style={'width': '90%', 'maxWidth': '500px', 'margin': '0 auto 20px auto', 'backgroundColor': THEME['container_bg'], 'border': f"1px solid {THEME['container_border']}", 'borderRadius': '12px', 'backdropFilter': 'blur(10px)', 'padding': '5px'}),

    html.P(id='prediction-summary-text', style={'textAlign': 'center', 'padding': '10px 0', 'fontSize': '18px', 'color': THEME['text']}),
    html.Div(id='graph-container', style={'height': '65vh', 'width': '98%', 'margin': 'auto', 'borderRadius': '15px', 'boxShadow': '0 0 25px 5px rgba(255, 255, 255, 0.15)'}),
    html.Div(id='info-panels-container')
])


# --- Main Callback to Generate Content ---
@app.callback(
    [Output('graph-container', 'children'),
     Output('info-panels-container', 'children'),
     Output('prediction-summary-text', 'children')],
    Input('ticker-dropdown', 'value')
)
def update_layout_and_data(selected_ticker):

    global processed_data_df, source_data_df
    processed_data_df, source_data_df = process_and_score_stocks(
        six_month_spearman_lagged_correlations, three_month_spearman_lagged_correlations, screener_data_df,
        selected_ticker, min_nodes, max_nodes, threshold_percent
    )

    if processed_data_df.empty or source_data_df.empty:
        empty_fig = go.Figure().update_layout(title=f"Data not available for {selected_ticker}", title_x=0.5, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font_color='white')
        return dcc.Graph(id='network-graph', figure=empty_fig, style={'height': '100%'}), html.Div(f"Data not available for {selected_ticker}", style={'textAlign': 'center', 'padding': '20px'}), ""

    graph = dcc.Graph(id='network-graph', figure=solar_system_visual(selected_ticker, processed_data_df, source_data_df, screener_data_df), style={'height': '100%'})

    # --- Generate data for panels and summary text ---
    star_info_screener = screener_data_df[screener_data_df['code'] == selected_ticker].iloc[0]
    star_info_source = source_data_df[source_data_df['ticker'] == selected_ticker].iloc[0]
    grav_impact = star_info_source.get('gravitational_impact', 0)

    # Summary text
    source_name = star_info_screener.get('name', selected_ticker)
    direction = "increase" if grav_impact >= 0 else "decrease"
    strength = grav_impact
    prediction_summary = f"{source_name} ({selected_ticker}) is predicted to {direction} today with a prediction strength of {strength:.2f}%."

    # Star Info Panel
    star_color = get_node_color(grav_impact, -80, 80)
    star_image_src = create_model_image_svg(star_color, subdivisions=2, texture_spots_count=10)
    star_text_content = html.Div([
        html.P(f"Name: {star_info_screener['name']} ({selected_ticker})", style={'margin': '0 0 5px 0'}),
        html.P(f"Market Cap: ${star_info_screener.get('market_capitalization', 0)/1e12:.2f}T", style={'margin': '0 0 5px 0'}),
        html.P(f"Gravitational Impact: {grav_impact:.2f}%", style={'margin': '0'}),
    ])
    star_info_panel = html.Div([
        html.H3("Star Information", style=header_style),
        html.Div([
            html.Img(src=star_image_src, style={'height': '150px', 'width': '150px', 'marginRight': '20px'}),
            star_text_content
        ], style={'display': 'flex', 'alignItems': 'center'})
    ], style=container_style)

    # --- Predictions Panel ---
    prediction_items = []
    if 'top_positive_impacts' in globals() and not top_positive_impacts.empty and 'top_negative_impacts' in globals() and not top_negative_impacts.empty:
        combined_impacts = pd.concat([top_positive_impacts.head(5), top_negative_impacts.head(5)])
        prediction_items = [
            html.Div([
                html.Span(row['ticker']),
                html.Span(f"{row['gravitational_impact']:.2f}%", style={'color': '#4ade80' if row['gravitational_impact'] > 0 else '#f87171', 'fontWeight': 'bold'})
            ], style={'display': 'flex', 'justifyContent': 'space-between', 'padding': '8px 0', 'borderBottom': f'1px solid {THEME["container_border"]}'})
            for _, row in combined_impacts.iterrows()
        ]
    else:
         prediction_items = [html.Div("Top predictions not available.")]
    predictions_panel = html.Div([html.H3("Top Predictions", style=header_style), html.Div(prediction_items)], style=container_style)

    # --- Legend Panel ---
    legend_gradient_style = {'height': '15px', 'width': '100%', 'maxWidth': '300px', 'borderRadius': '5px', 'marginBottom': '8px', 'background': f"linear-gradient(to right, {RED_SPECTRUM['dark']}, {RED_SPECTRUM['light']} 49.9%, {GREEN_SPECTRUM['light']} 50.1%, {GREEN_SPECTRUM['dark']})"}
    legend_content = html.Div([
        html.Div(style=legend_gradient_style),
        html.Div([html.Span("Decrease"), html.Span("Increase")], style={'display': 'flex', 'justifyContent': 'space-between', 'fontSize': '12px', 'width': '100%', 'maxWidth': '300px'})
    ], style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center'})
    legend_panel_container = html.Div([
        html.H3("Legend", style={**header_style, 'marginBottom': '15px', 'textAlign': 'center'}),
        legend_content
    ], style={**container_style, 'padding': '15px 20px', 'width': '100%', 'align-self': 'center'})

    # --- Planet Info Table ---
    planets_df = processed_data_df[processed_data_df['source'] == selected_ticker].copy()
    table_header = [html.Thead(html.Tr([html.Th(col, style={'padding': '12px', 'textAlign': 'left', 'borderBottom': f"2px solid {THEME['container_border']}"}) for col in ["Ticker", "Name", "Market Cap", "Daily Change", "Correlation", "Grav. Force"]]))]
    table_rows = []
    if not planets_df.empty:
        for _, p_row in planets_df.iterrows():
            s_info = screener_data_df[screener_data_df['code'] == p_row['target']].iloc[0]
            planet_color = get_node_color(p_row['Daily Change'], -5, 5)
            planet_image_src = create_model_image_svg(planet_color, subdivisions=2, texture_spots_count=5)

            ticker_cell = html.Div([
                html.Img(src=planet_image_src, style={'height': '40px', 'width': '40px', 'marginRight': '10px'}),
                html.Span(p_row['target'])
            ], style={'display': 'flex', 'alignItems': 'center'})

            table_rows.append(html.Tr([
                html.Td(ticker_cell, style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(s_info['name'], style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(f"${s_info['market_capitalization']/1e9:.2f}B", style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(f"{p_row['Daily Change']:.2f}%", style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(f"{p_row['unified_correlation']:.3f}", style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
                html.Td(f"{p_row['signed_gravitational_force']:.2f}", style={'padding': '8px 12px', 'borderBottom': f'1px solid {THEME["container_border"]}'}),
            ]))

    # Dynamically set the height of the table container
    table_wrapper_style = {'overflowX': 'auto'}
    if len(planets_df) > 5:
        table_wrapper_style['maxHeight'] = '300px'
        table_wrapper_style['overflowY'] = 'auto'

    planet_table_panel = html.Div([
        html.H3("Planet Information", style=header_style),
        html.Div(html.Table(table_header + [html.Tbody(table_rows)], style={'width': '100%', 'borderCollapse': 'collapse'}), style=table_wrapper_style)
    ], style=container_style)

    # Assemble the info panels layout
    info_panels = html.Div([
        legend_panel_container,
        star_info_panel,
        planet_table_panel,
        predictions_panel
    ], style={'display': 'flex', 'flexDirection': 'column', 'gap': '20px', 'padding': '20px 0'})

    return graph, info_panels, prediction_summary

# --- Run the App ---
if __name__ == '__main__':
    app.run(debug=True, port=8051)


<IPython.core.display.Javascript object>